In [1]:
# 어플리케이션 패키지
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import requests 
from bs4 import BeautifulSoup

import datetime
from time import sleep
import json
from openpyxl import Workbook, load_workbook
import unicodedata
import re

In [2]:
BASE_URL = 'https://www.instagram.com/'
LOGIN_URL = BASE_URL + 'accounts/login/'
LIKER_URL = BASE_URL + 'graphql/query/?query_hash=d5d763b1e2acf209d62d22d184488e57'
COMMENT_URL = BASE_URL + 'graphql/query/?query_hash=bc3296d1ce80a24b1b6e40b1e72903f5'
FOLLOWER_URL = BASE_URL + 'graphql/query/?query_hash=37479f2b8209594dde7facb0d904896a'

## 엑셀 파일 저장

In [3]:
NOW = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
FILE_PATH = './output/'
FILE_NAME = f'17년생아들.xlsx'

In [4]:
# 엑셀 파일 생성
def create_excel(crawl_type):    
    wb = Workbook()
    ws = wb.active

#     wb.create_sheet('좋아요', 0)
#     wb.create_sheet('댓글', 1)
#     ws_liker = wb['좋아요']
#     ws_comment = wb['댓글']

    title = ''
    if crawl_type == 'like':
        title = ['번호','사용자ID','사용자명']
    elif crawl_type == 'comment':
        title = ['번호','레벨','사용자ID','내용','작성일시']
    elif crawl_type == 'tag':
        title = ['번호','사용자ID','사용자명','팔로워 수','팔로잉 수']
    elif crawl_type == 'follower':
        title = ['번호','사용자ID','사용자명']        
        
    for kwd, j in zip(title, list(range(1, len(title) + 1))):
        ws.cell(row=1, column=j).value = kwd
        
    wb.save(FILE_PATH + FILE_NAME)

In [5]:
crawl_type = 'tag'
create_excel(crawl_type)

In [6]:
# 엑셀 입력
def write_excel(append_row):
    wb = load_workbook(FILE_PATH + FILE_NAME, data_only=True)
    ws = wb.active
    
#     if crawl_type == 'likers':
#         ws = wb['좋아요']
#     elif crawl_type == 'comments':
#         ws = wb['댓글']

    ws.append(append_row)
    wb.save(FILE_PATH + FILE_NAME)

In [7]:
def get_user_by_user_id(user_id, headers):
    base_url = "https://i.instagram.com/api/v1/users/web_profile_info/?username={}"
    if user_id:
        try:
            res = requests.get(base_url.format(user_id), headers=headers)
            user_info = res.json()
        except Exception as e:
            print("getting user failed, due to '{}'".format(e.message))
    return user_info['data']

In [8]:
from seleniumwire import webdriver as webdriver_wire
from seleniumwire.utils import decode
from selenium.webdriver.common.keys import Keys

In [8]:
# browser = webdriver_wire.Chrome(service=Service(ChromeDriverManager().install()))
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7.46M/7.46M [00:00<00:00, 50.4MB/s]


## 로그인

In [9]:
def login_instagram(browser, id, password):
    browser.get(LOGIN_URL)
    sleep(5)
    inputs = browser.find_elements(by=By.TAG_NAME, value='input')
    inputs[0].clear()
    inputs[1].clear()
    inputs[0].send_keys(id)
    inputs[1].send_keys(password)
    inputs[1].submit()
    sleep(3)

    if '잘못된 비밀번호' in browser.page_source:
        return 'PWERR'
    elif '입력한 사용자 이름' in browser.page_source:
        return 'IDERR'
    else:
        return 'OK'

In [10]:
id = '01042170052'
pw = 'smc311125!'
login_instagram(browser, id, pw)

'OK'

In [69]:
# 헤더 생성(수작업으로 변경)
headers = {
        "cookie": 'ig_did=D2445F7C-DE67-412F-AB9F-C8D7C56D9EE0; ig_nrcb=1; mid=YuTXIQAEAAF3RwYLTX8-GwA4htdW; datr=AD7lYuirfwZ6reHB2T5jHaXV; fbm_124024574287414=base_domain=.instagram.com; shbid="4632\0547429511270\0541695112675:01f7d6e1dec8ffd7c9d0e8955c6652e2e07fd083f416db96eed2e84ec1459dde0c9b8b55"; shbts="1663576675\0547429511270\0541695112675:01f77314e108526414d87c507cea9b611ae5e67891fed8ee4888d37bd3c973cc73a6ed31"; fbsr_124024574287414=F8KFOHBCRR5ss6-fRck3-9HhA2rWcHCF0S8JLTCDtiA.eyJ1c2VyX2lkIjoiMTAwMDg0MTM3ODk5MDc1IiwiY29kZSI6IkFRQlVUMHVVSEprQVUtWjR4VDNOZlpOM1RlYWMyZ0FtaDNveTRadTNNbWx2djRNbFR5RzJuZlZtZDNyS3d5MGFHbEMxS3AzX3ZqWDVJSDdzcWc4dXVBeDJ0b2FaMUFuWlBiQkdGakkySFFxdnJndTExbHZhTjNoaU85SExleFIwdmZXSHZwVUJoWUlEMVZxc0QtbWRqZDIyaVhsMjJtbU15OHNnc2M4WVJYZ09FTW12a1VCODJBTVlnU1ZVejNmMnlyUU1WaTRHQkMwQW9LRllCMFU0M1dkZHQwV3RURFQzY1NhZEZhYmpXR2lNUTJHVzY5M0VZdXM2eXZzWXpmSFp3a0lFREk5VFhYb2E5NHp6M0VXemZmMnBHQnpydUJQOUdxVzlnMThqeTJHRGlPWjFrVU9nei1PTW1hT1VIemM2X2ZMbUp3eWI1MzhsS1ZKWUd1OG9mRmJKIiwib2F1dGhfdG9rZW4iOiJFQUFCd3pMaXhuallCQVAxZWNqSUFmd0YyNkt3UXJuRERlRDE2WTM1T3VVTkFtOHJSeFpBVzhlSGpoUlNqNTB0YkU0ZmxaQ3RaQVhEbHZqSzZMZlVqN2dSdllvdkdkZ2FYcENEWHhsVXdxV1F5cEw3ZUhaQVA0SnIwVE9ramdXR0tocVBtWkJOWWxaQXBzSW5mbXhIZ0lvcHFYOUFxTVpCUFFBOTQ4OG9LQ0g0TkxTTXc4Y3dTZkJsTGhXd2txZ283RVlaRCIsImFsZ29yaXRobSI6IkhNQUMtU0hBMjU2IiwiaXNzdWVkX2F0IjoxNjYzNzMzNjU1fQ; dpr=2; fbsr_124024574287414=F8KFOHBCRR5ss6-fRck3-9HhA2rWcHCF0S8JLTCDtiA.eyJ1c2VyX2lkIjoiMTAwMDg0MTM3ODk5MDc1IiwiY29kZSI6IkFRQlVUMHVVSEprQVUtWjR4VDNOZlpOM1RlYWMyZ0FtaDNveTRadTNNbWx2djRNbFR5RzJuZlZtZDNyS3d5MGFHbEMxS3AzX3ZqWDVJSDdzcWc4dXVBeDJ0b2FaMUFuWlBiQkdGakkySFFxdnJndTExbHZhTjNoaU85SExleFIwdmZXSHZwVUJoWUlEMVZxc0QtbWRqZDIyaVhsMjJtbU15OHNnc2M4WVJYZ09FTW12a1VCODJBTVlnU1ZVejNmMnlyUU1WaTRHQkMwQW9LRllCMFU0M1dkZHQwV3RURFQzY1NhZEZhYmpXR2lNUTJHVzY5M0VZdXM2eXZzWXpmSFp3a0lFREk5VFhYb2E5NHp6M0VXemZmMnBHQnpydUJQOUdxVzlnMThqeTJHRGlPWjFrVU9nei1PTW1hT1VIemM2X2ZMbUp3eWI1MzhsS1ZKWUd1OG9mRmJKIiwib2F1dGhfdG9rZW4iOiJFQUFCd3pMaXhuallCQVAxZWNqSUFmd0YyNkt3UXJuRERlRDE2WTM1T3VVTkFtOHJSeFpBVzhlSGpoUlNqNTB0YkU0ZmxaQ3RaQVhEbHZqSzZMZlVqN2dSdllvdkdkZ2FYcENEWHhsVXdxV1F5cEw3ZUhaQVA0SnIwVE9ramdXR0tocVBtWkJOWWxaQXBzSW5mbXhIZ0lvcHFYOUFxTVpCUFFBOTQ4OG9LQ0g0TkxTTXc4Y3dTZkJsTGhXd2txZ283RVlaRCIsImFsZ29yaXRobSI6IkhNQUMtU0hBMjU2IiwiaXNzdWVkX2F0IjoxNjYzNzMzNjU1fQ; csrftoken=iCT1CLsBsL4b7lJcinUnlg5lTpWFzwgN; ds_user_id=5347098824; sessionid=5347098824%3AVIx7slcBZ9SFsM%3A17%3AAYc1HL5yW_AEC0SuImzWcTwfIngNrXqTX7s9K3viNg; rur="VLL\0545347098824\0541695269712:01f722b25b329d866c8cc75ed5ee31070ca5c9a82450229265e6231b2493acbc15573cc6"'
        , "referer" : "https://www.instagram.com/"
        , "x-asbd-id": "198387"
        , "x-ig-app-id": "936619743392459"
        , "x-csrftoken": "iCT1CLsBsL4b7lJcinUnlg5lTpWFzwgN"
        , "x-ig-www-claim": "hmac.AR1ToUQmxO7ELz9SlvFwj4fs2Jmq4X7uZ_UHKMIwzbx6x3h2"
        , "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }

In [ ]:
2928581054815872371

In [ ]:
2925701864416431206

In [ ]:
{"items":[{"taken_at":1662990862,"pk":"2925701864416431206","id":"2925701864416431206_3273209062","device_timestamp":1662990266433091,"media_type":8,"code":"CiaLlg7PUxm","client_cache_key":"MjkyNTcwMTg2NDQxNjQzMTIwNg==.2","filter_type":0,"is_unified_video":false,"should_request_ads":false,"caption_is_edited":true,"like_and_view_counts_disabled":false,"commerciality_status":"not_commercial","is_paid_partnership":false,"is_visual_reply_commenter_notice_enabled":true,"original_media_has_visual_reply_media":false,"has_delayed_metadata":false,"comment_likes_enabled":true,"comment_threading_enabled":true,"has_more_comments":true,"max_num_visible_preview_comments":2,"preview_comments":[],"comments":[],"can_view_more_preview_comments":false,"comment_count":1,"commenting_disabled_for_viewer":true,"hide_view_all_comment_entrypoint":false,"inline_composer_display_condition":"never","carousel_media_count":9,"carousel_media":[{"id":"2925701857353332258_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT8AT_L3H0pP-IrBvpLIeNpTOAziN14SSszqlMnBF4a47w\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT_L0itnzEVDJVNnTTQd0g1SpmtelWJTdxKC0759X_o8UA\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT8abUbs9pEvCyKnENCc9am8j2uZ4BdlQTAKDw22PiZ1Sw\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT9x4OQ1UXwp1Lh7viCleu8hlvI6G8-KLJyBczyac5EkGA\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT_Lt9aOxgcS1gCtMLZWnt7ksTStLFRFmZgyRCxdTTVq7g\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT8nwxeIIDeOkDJAXktTQQLoD70gQT-7Xmea2yvrwJ7W0w\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT86olXuVC45lG7G8bfrwqjEKxdbh-gi9stD4pLHxmWbSg\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT_L0itnzEVDJVNnTTQd0g1SpmtelWJTdxKC0759X_o8UA\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT8abUbs9pEvCyKnENCc9am8j2uZ4BdlQTAKDw22PiZ1Sw\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT9x4OQ1UXwp1Lh7viCleu8hlvI6G8-KLJyBczyac5EkGA\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT_Lt9aOxgcS1gCtMLZWnt7ksTStLFRFmZgyRCxdTTVq7g\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT8nwxeIIDeOkDJAXktTQQLoD70gQT-7Xmea2yvrwJ7W0w\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT86olXuVC45lG7G8bfrwqjEKxdbh-gi9stD4pLHxmWbSg\u0026oe=63340C54\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306055688_825831735078979_4480403280474234743_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=tp7n0IksYpIAX9i3zK3\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM1MzMzMjI1OA%3D%3D.2-ccb7-5\u0026oh=00_AT-VKez-n-vrfKSYLb9l2PChTNVSIRmfqWo9-MFM8fsYmg\u0026oe=63340C54\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of 1 person, standing and outdoors.","pk":"2925701857353332258","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925701857235861836_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT8V0nbULHMXVJJ7ZvKXOCnmSI2bA5OwBkor-C_ptl8SEA\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT_VKno0rG30otdGWUivPwo9VlF01Xh_QEfbEEghrxm2FQ\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT9auA2hg_XM69Q7uhZ6PL9rsamCEd_Ilct9JcdxdsbCPg\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT8smIIJ43U_YE_bjDVQqo2i3NhZRHO1rYKGeZxfjRdQ1w\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT8dWcFxDqJjy4aqF0sQJudxGbfzdcfu9-JRhwLmBxW4ng\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT9B67QuwvUxapeCcyGezGNKTJm_PU01MBi0qTSXgiYRWQ\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT-mG2yT9qBWEZkPkFbUf6IH8xkDuKMeYenauitKdE77Bw\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT_VKno0rG30otdGWUivPwo9VlF01Xh_QEfbEEghrxm2FQ\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT9auA2hg_XM69Q7uhZ6PL9rsamCEd_Ilct9JcdxdsbCPg\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT8smIIJ43U_YE_bjDVQqo2i3NhZRHO1rYKGeZxfjRdQ1w\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT8dWcFxDqJjy4aqF0sQJudxGbfzdcfu9-JRhwLmBxW4ng\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT9B67QuwvUxapeCcyGezGNKTJm_PU01MBi0qTSXgiYRWQ\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT-mG2yT9qBWEZkPkFbUf6IH8xkDuKMeYenauitKdE77Bw\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306151512_1253907755425875_4069105121356328452_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=cB6JmYzBrkcAX9SMR1I\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg2MTgzNg%3D%3D.2-ccb7-5\u0026oh=00_AT_Aaul-OvkCeHj67vqh-xEDTeZf7t4iDzsX4pVnFzNMDA\u0026oe=6332F3F4\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of food.","pk":"2925701857235861836","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925701857361633570_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT-Mf-gin3PUw7pmEDFrEUj9UBuKZD6ooQm9WZWelHHzwA\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT8d3Rc7_pJinv2MPARdTfl_KQ2IhAy2KkZk7E7Q86lTZw\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT_U1C7drQmkEKdahxHdnzsNnaUxR133u8Oemdr8GUvNOw\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT-wflDUAiic7aSq3MJKzfKUtbeLnNvZVPiHjfslIjOCBg\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT8GIBqc-KRDq5QP4ECWd35edVrhMv_IHR9hJkblE5Rqxg\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT8VI11PQEAjPQfs2f-ONPUZi1LBVKT9El7YkObLC3_RrA\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT_iZDl07oS68dMF-1KEVlPaAoadnn88Jccnw0f7Auww9A\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT8d3Rc7_pJinv2MPARdTfl_KQ2IhAy2KkZk7E7Q86lTZw\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT_U1C7drQmkEKdahxHdnzsNnaUxR133u8Oemdr8GUvNOw\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT-wflDUAiic7aSq3MJKzfKUtbeLnNvZVPiHjfslIjOCBg\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT8GIBqc-KRDq5QP4ECWd35edVrhMv_IHR9hJkblE5Rqxg\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT8VI11PQEAjPQfs2f-ONPUZi1LBVKT9El7YkObLC3_RrA\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT_iZDl07oS68dMF-1KEVlPaAoadnn88Jccnw0f7Auww9A\u0026oe=633305AD\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/305988156_3434817730079565_5763574603112076948_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=c373WzaYq3YAX_u6Zzj\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTYzMzU3MA%3D%3D.2-ccb7-5\u0026oh=00_AT_d3JgS99m87fa9qBA5F5wWfcI5Abuw7MOLGbM8i2liYw\u0026oe=633305AD\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of 2 people, child, people standing and outdoors.","pk":"2925701857361633570","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925701857252734510_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT8eu4E2inEu2bdTTzuOjWnOlPKmKlm2HauwmQt7LDBt7A\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT8w4ENyQlJYWVLsCOLg0W3AjFBYNOCUcI7cEiII4yxHMg\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT-A2yClh4yp1lSojw6KUPghOPt-jpndwI_irnnd1-It2A\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT9I63LcSgRpMNgPOVWUMr-8uKAmQKhTe15ZBSwKMI-dSQ\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT-o3Ev_FgKS9Iv3aZXGt8x38RsTl5Sl3H5BEAN2yKIlYg\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT8Sz-nDZBE6IHXWBQr4AaBpvu5dZiGFRb9hLF5XM0Xq8g\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT805X8LfN8fGYI9dxEOQwZue98SQTMBu_ebhl63KqvQAQ\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT8w4ENyQlJYWVLsCOLg0W3AjFBYNOCUcI7cEiII4yxHMg\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT-A2yClh4yp1lSojw6KUPghOPt-jpndwI_irnnd1-It2A\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT9I63LcSgRpMNgPOVWUMr-8uKAmQKhTe15ZBSwKMI-dSQ\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT-o3Ev_FgKS9Iv3aZXGt8x38RsTl5Sl3H5BEAN2yKIlYg\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT8Sz-nDZBE6IHXWBQr4AaBpvu5dZiGFRb9hLF5XM0Xq8g\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT805X8LfN8fGYI9dxEOQwZue98SQTMBu_ebhl63KqvQAQ\u0026oe=6333FD02\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306566608_1640983416299145_8682578852994483235_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=109\u0026_nc_ohc=KYgVlgeSVPYAX8dPTHR\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzI1MjczNDUxMA%3D%3D.2-ccb7-5\u0026oh=00_AT_SgtDemR9020AGMGIPzXhea3_VOss_XeGBQDy9xQ-Zmw\u0026oe=6333FD02\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of 1 person, standing, sky and tree.","pk":"2925701857252734510","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925696868660816884_3273209062","media_type":2,"image_versions2":{"candidates":[{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s640x640\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT8FZVfENyPCtoTeLZoiSZqHuJsT5sBHHHlc3FADRvgGDA\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT_5XUqJkgg-tgvgwBStXDzHkIQ_T-5r0b_f-FParHO_tQ\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT_exo1SZJ_bh5fud6zVrAmIRExoGVcrFCrSmjOjy6H6Bw\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT_2TWPf21IqrCFPTlWXT7o926K1M4-mImf6kb0ay92ETQ\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT-39FqyOYxkBU9yPbIaUbXizVuzwkqH6wXhqLEYPdFBFw\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT-39FqyOYxkBU9yPbIaUbXizVuzwkqH6wXhqLEYPdFBFw\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s640x640\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT8FZVfENyPCtoTeLZoiSZqHuJsT5sBHHHlc3FADRvgGDA\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT_5XUqJkgg-tgvgwBStXDzHkIQ_T-5r0b_f-FParHO_tQ\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT_exo1SZJ_bh5fud6zVrAmIRExoGVcrFCrSmjOjy6H6Bw\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT_2TWPf21IqrCFPTlWXT7o926K1M4-mImf6kb0ay92ETQ\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306288050_1034742897223636_9022694338779218306_n.jpg?stp=dst-jpg_e15_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=aYhSzIGUQVMAX8G9RCD\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg2ODY2MDgxNjg4NA%3D%3D.2-ccb7-5\u0026oh=00_AT8rL-aZLwutptzWHoSaUY4tOTJe4tQS5SxhdXbdyZspPQ\u0026oe=6334BAA7\u0026_nc_sid=30a2ef"}]},"original_width":720,"original_height":720,"video_versions":[{"type":101,"width":720,"height":720,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306629004_478296427501417_2252606679736944371_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjcyMC5jYXJvdXNlbF9pdGVtLmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=101\u0026_nc_ohc=0aUDk4eXWwEAX_0QLHA\u0026edm=ALQROFkBAAAA\u0026vs=485608309795180_3242921090\u0026_nc_vs=HBkcFQAYJEdJekpSaEpwbDZnUUFyTUJBUE9HWkNUWDNVSWZia1lMQUFBRhUAAsgBACgAGAAbAYgHdXNlX29pbAEwFQAAJqD64rHgobQ%2FFQIoAkMzLBdAJjMzMzMzMxgSZGFzaF9iYXNlbGluZV8xX3YxEQB17gcA\u0026ccb=7-5\u0026oe=632F2F08\u0026oh=00_AT_c-zO1SUkZa1Ku6MHyYwXtte1G_GqeZjNjnnHSOo5Ocg\u0026_nc_sid=30a2ef","id":"485608309795180v"},{"type":102,"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306877730_1049622232422835_32950459802269500_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjQ4MC5jYXJvdXNlbF9pdGVtLmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=bGTCT6EBVxAAX_U7Rk6\u0026edm=ALQROFkBAAAA\u0026vs=494011655469131_350175382\u0026_nc_vs=HBkcFQAYJEdDS1ZTaEt6MlU4NG9Mb0RBRHczczhORUVIVUFia1lMQUFBRhUAAsgBACgAGAAbAYgHdXNlX29pbAEwFQAAJqD64rHgobQ%2FFQIoAkMzLBdAJjMzMzMzMxgSZGFzaF9iYXNlbGluZV8yX3YxEQB17gcA\u0026ccb=7-5\u0026oe=632F6E29\u0026oh=00_AT9DC90f_1xZpjIY6j21qZhMysnLQfb_9d7RLNh6ob96Mw\u0026_nc_sid=30a2ef","id":"494011655469131v"},{"type":103,"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306877730_1049622232422835_32950459802269500_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjQ4MC5jYXJvdXNlbF9pdGVtLmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=bGTCT6EBVxAAX_U7Rk6\u0026edm=ALQROFkBAAAA\u0026vs=494011655469131_350175382\u0026_nc_vs=HBkcFQAYJEdDS1ZTaEt6MlU4NG9Mb0RBRHczczhORUVIVUFia1lMQUFBRhUAAsgBACgAGAAbAYgHdXNlX29pbAEwFQAAJqD64rHgobQ%2FFQIoAkMzLBdAJjMzMzMzMxgSZGFzaF9iYXNlbGluZV8yX3YxEQB17gcA\u0026ccb=7-5\u0026oe=632F6E29\u0026oh=00_AT9DC90f_1xZpjIY6j21qZhMysnLQfb_9d7RLNh6ob96Mw\u0026_nc_sid=30a2ef","id":"494011655469131v"}],"video_duration":11.1,"is_dash_eligible":1,"video_dash_manifest":"\u003cMPD xmlns=\"urn:mpeg:dash:schema:mpd:2011\" minBufferTime=\"PT1.500S\" type=\"static\" mediaPresentationDuration=\"PT0H0M11.100S\" maxSegmentDuration=\"PT0H0M5.000S\" profiles=\"urn:mpeg:dash:profile:isoff-on-demand:2011,http://dashif.org/guidelines/dash264\"\u003e\n \u003cPeriod duration=\"PT0H0M11.100S\"\u003e\n  \u003cAdaptationSet segmentAlignment=\"true\" maxWidth=\"720\" maxHeight=\"720\" maxFrameRate=\"30\" par=\"1:1\" lang=\"und\" subsegmentAlignment=\"true\" subsegmentStartsWithSAP=\"1\"\u003e\n   \u003cRepresentation id=\"485608309795180vd\" mimeType=\"video/mp4\" codecs=\"avc1.4D401E\" width=\"720\" height=\"720\" frameRate=\"30\" sar=\"1:1\" startWithSAP=\"1\" bandwidth=\"1755817\" FBQualityClass=\"hd\" FBQualityLabel=\"720w\" FBPlaybackResolutionMos=\"0:100.00,1080:52.19\"\u003e\n    \u003cBaseURL urlExpiration=\"1664036616\"\u003ehttps://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306629004_478296427501417_2252606679736944371_n.mp4?_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026amp;_nc_cat=101\u0026amp;_nc_ohc=0aUDk4eXWwEAX_0QLHA\u0026amp;edm=ALQROFkBAAAA\u0026amp;ccb=7-5\u0026amp;oh=00_AT9yFnyL9hS44WU1yNhGc9EmybpLiu-szHFydE6j5U2JlQ\u0026amp;oe=632F2F08\u0026amp;_nc_sid=30a2ef\u003c/BaseURL\u003e\n    \u003cSegmentBase indexRangeExact=\"true\" indexRange=\"840-919\" FBFirstSegmentRange=\"920-495345\" FBSecondSegmentRange=\"495346-932402\" FBPrefetchSegmentRange=\"920-932402\"\u003e\n      \u003cInitialization range=\"0-839\"/\u003e\n    \u003c/SegmentBase\u003e\n   \u003c/Representation\u003e\n  \u003cRepresentation id=\"494011655469131v\" mimeType=\"video/mp4\" codecs=\"avc1.42C01E\" width=\"480\" height=\"480\" frameRate=\"30\" sar=\"1:1\" startWithSAP=\"1\" bandwidth=\"864769\" FBQualityClass=\"sd\" FBQualityLabel=\"480w\" FBPlaybackResolutionMos=\"0:100.00,1080:22.09\"\u003e\n    \u003cBaseURL urlExpiration=\"1664052777\"\u003ehttps://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306877730_1049622232422835_32950459802269500_n.mp4?_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026amp;_nc_cat=107\u0026amp;_nc_ohc=bGTCT6EBVxAAX_U7Rk6\u0026amp;edm=ALQROFkBAAAA\u0026amp;ccb=7-5\u0026amp;oh=00_AT-EVj1B8oayvsvmmHmrvAWNmB1IvOOBUcmtBrwMD8AS7w\u0026amp;oe=632F6E29\u0026amp;_nc_sid=30a2ef\u003c/BaseURL\u003e\n    \u003cSegmentBase indexRangeExact=\"true\" indexRange=\"838-917\" FBFirstSegmentRange=\"918-277824\" FBSecondSegmentRange=\"277825-475655\" FBPrefetchSegmentRange=\"918-475655\"\u003e\n      \u003cInitialization range=\"0-837\"/\u003e\n    \u003c/SegmentBase\u003e\n   \u003c/Representation\u003e\n  \u003c/AdaptationSet\u003e\n \u003c/Period\u003e\n\u003c/MPD\u003e","video_codec":"avc1.4D401E","number_of_qualities":2,"pk":"2925696868660816884","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925701857361683172_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT9uRI6LPxY3tAWeuzBysgN_57YQzMuM9bYXGJJxueaJFQ\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT-Gdmnwzohm2wuMhwM-dVKLuC64orAQO9XCVpmgdOoH9A\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT_PoPX3RAborH0l4iW8A_CCMRJY-zSpK2Lh420nhDOZog\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT8hxN7Ldl4MfgTPHD8CMhrF2ne-WJ_9i8Fy6Mcu-fvhPg\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT_dESD3rfrGtfmxfI5k3fiE913Lea2Vct_Tq5koB73UTg\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT-FW475icTIS7-7u1-FepRfAZR9um7qY1OqsEfxHUsP4g\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT-iEWB5tXCJ9b5SWfoYpO6EvtQvNSPajAIOd1c6r6RgOg\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT-Gdmnwzohm2wuMhwM-dVKLuC64orAQO9XCVpmgdOoH9A\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT_PoPX3RAborH0l4iW8A_CCMRJY-zSpK2Lh420nhDOZog\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT8hxN7Ldl4MfgTPHD8CMhrF2ne-WJ_9i8Fy6Mcu-fvhPg\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT_dESD3rfrGtfmxfI5k3fiE913Lea2Vct_Tq5koB73UTg\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT-FW475icTIS7-7u1-FepRfAZR9um7qY1OqsEfxHUsP4g\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT-iEWB5tXCJ9b5SWfoYpO6EvtQvNSPajAIOd1c6r6RgOg\u0026oe=633357C0\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306652004_1542889209461383_2751893170779693424_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=107\u0026_nc_ohc=7CSB_jGK1gIAX_jN6a2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzM2MTY4MzE3Mg%3D%3D.2-ccb7-5\u0026oh=00_AT8Du8YIaLUU8nZ-Z-BeCwp2TyqP5wXPArn7-c2oSzorxA\u0026oe=633357C0\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of 1 person, standing, outdoors and text that says '백'.","pk":"2925701857361683172","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925701857235890453_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT-uLfHHIgYq7gSTcB7sRzaY1EaSLRiHzwiznIpxEJ479g\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT_2mbEux7pPgX6_CVLQqviVKZewLd4BO03ySt03g060yw\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT9gfDXtQnaUHt30H0722DZ_asuGkFh5xzETP4hcF2NVsw\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT9yt4LCDleoCZpuP8clmtcgKQYt2dXwyWUbXUBka1x_dA\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT-hAU_7l8lnHqI6cMlXtb3OXsd6-fBWBz1nde2u64ZwUw\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT__yzN9l3qjMZNvXIVzK3JjchbJ-xtn_pX0WLT9bneJQA\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT85rMuDEgsaZaEQbSA7O7ZTeLmies5UQUxHNIHtQ5sW5Q\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT_2mbEux7pPgX6_CVLQqviVKZewLd4BO03ySt03g060yw\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT9gfDXtQnaUHt30H0722DZ_asuGkFh5xzETP4hcF2NVsw\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT9yt4LCDleoCZpuP8clmtcgKQYt2dXwyWUbXUBka1x_dA\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT-hAU_7l8lnHqI6cMlXtb3OXsd6-fBWBz1nde2u64ZwUw\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT__yzN9l3qjMZNvXIVzK3JjchbJ-xtn_pX0WLT9bneJQA\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT85rMuDEgsaZaEQbSA7O7ZTeLmies5UQUxHNIHtQ5sW5Q\u0026oe=63347CA6\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306197880_593290969002143_4090373287276383938_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=111\u0026_nc_ohc=ozeen6u-x6sAX_W5tI2\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzIzNTg5MDQ1Mw%3D%3D.2-ccb7-5\u0026oh=00_AT9Es0a5jg-x-ea1GRTpqSyJ9eznywOaQ2ggQkrjQDe-xQ\u0026oe=63347CA6\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of 2 people, people sitting and body of water.","pk":"2925701857235890453","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925696872427300223_3273209062","media_type":2,"image_versions2":{"candidates":[{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s640x640\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT-nyC_rSYKF6WpouyIKFbmVysbBmVj7XMjY8KLGbweeKA\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT_M3-ukBSSjLfGit7ZZCE3KaHu3tsUMWtKvbHBvNjfV0Q\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT-RjdeCB_H9u88VT0W3Ad0xZu4SE4b_Jk_0zvvJPYblZQ\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT_kVtD0Nn6UoalLUy8RErzOFtO-Y4l4mMwQFo2HTWYo8g\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT8_cVnMkX8lJlPzip28E1naDjezFphCtGpzCy10bHGt_Q\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT8_cVnMkX8lJlPzip28E1naDjezFphCtGpzCy10bHGt_Q\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s640x640\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT-nyC_rSYKF6WpouyIKFbmVysbBmVj7XMjY8KLGbweeKA\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT_M3-ukBSSjLfGit7ZZCE3KaHu3tsUMWtKvbHBvNjfV0Q\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT-RjdeCB_H9u88VT0W3Ad0xZu4SE4b_Jk_0zvvJPYblZQ\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT_kVtD0Nn6UoalLUy8RErzOFtO-Y4l4mMwQFo2HTWYo8g\u0026oe=63333412\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306584413_129930939566487_5255631403224959979_n.jpg?stp=dst-jpg_e15_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=Q3dVOIoHTb8AX8inRNQ\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTY5Njg3MjQyNzMwMDIyMw%3D%3D.2-ccb7-5\u0026oh=00_AT-ZJOCIE92n_-_EA3NHq4sUrjSHySZv5YX5tlGokq1Xdg\u0026oe=63333412\u0026_nc_sid=30a2ef"}]},"original_width":720,"original_height":720,"video_versions":[{"type":101,"width":720,"height":720,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306579382_403169795265464_6841618905692582694_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjcyMC5jYXJvdXNlbF9pdGVtLmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=108\u0026_nc_ohc=6FGTo62l15YAX91OiIU\u0026edm=ALQROFkBAAAA\u0026vs=616287853331251_2426043584\u0026_nc_vs=HBkcFQAYJEdMWUhSaEs0VTI1SXJtNEJBQ1l6WjNJbFVQSmVia1lMQUFBRhUAAsgBACgAGAAbAYgHdXNlX29pbAEwFQAAJuqst5zJ5eU%2FFQIoAkMzLBdAImZmZmZmZhgSZGFzaF9iYXNlbGluZV8xX3YxEQB17gcA\u0026ccb=7-5\u0026oe=632F8BE7\u0026oh=00_AT-I8BH43ovoqoGHOXETlcqxLu-NK7mJ_cAYfV24F-lbow\u0026_nc_sid=30a2ef","id":"616287853331251v"},{"type":102,"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306594272_1499246513860959_4781320869794637531_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjQ4MC5jYXJvdXNlbF9pdGVtLmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=lClKEJ6vijsAX-jIOJR\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026vs=1110030823239134_2605163082\u0026_nc_vs=HBkcFQAYJEdPQkJSaEpmc2VPSGpsTUZBTnRtUmFILXFWcENia1lMQUFBRhUAAsgBACgAGAAbAYgHdXNlX29pbAEwFQAAJuqst5zJ5eU%2FFQIoAkMzLBdAImZmZmZmZhgSZGFzaF9iYXNlbGluZV8yX3YxEQB17gcA\u0026ccb=7-5\u0026oe=632F488A\u0026oh=00_AT8pgczXJ5lh2__38_0RZlcnHsUSwuKqu_uUTthXBEZPJw\u0026_nc_sid=30a2ef","id":"1110030823239134v"},{"type":103,"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306594272_1499246513860959_4781320869794637531_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjQ4MC5jYXJvdXNlbF9pdGVtLmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=103\u0026_nc_ohc=lClKEJ6vijsAX-jIOJR\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026vs=1110030823239134_2605163082\u0026_nc_vs=HBkcFQAYJEdPQkJSaEpmc2VPSGpsTUZBTnRtUmFILXFWcENia1lMQUFBRhUAAsgBACgAGAAbAYgHdXNlX29pbAEwFQAAJuqst5zJ5eU%2FFQIoAkMzLBdAImZmZmZmZhgSZGFzaF9iYXNlbGluZV8yX3YxEQB17gcA\u0026ccb=7-5\u0026oe=632F488A\u0026oh=00_AT8pgczXJ5lh2__38_0RZlcnHsUSwuKqu_uUTthXBEZPJw\u0026_nc_sid=30a2ef","id":"1110030823239134v"}],"video_duration":9.2,"is_dash_eligible":1,"video_dash_manifest":"\u003cMPD xmlns=\"urn:mpeg:dash:schema:mpd:2011\" minBufferTime=\"PT1.500S\" type=\"static\" mediaPresentationDuration=\"PT0H0M9.200S\" maxSegmentDuration=\"PT0H0M5.000S\" profiles=\"urn:mpeg:dash:profile:isoff-on-demand:2011,http://dashif.org/guidelines/dash264\"\u003e\n \u003cPeriod duration=\"PT0H0M9.200S\"\u003e\n  \u003cAdaptationSet segmentAlignment=\"true\" maxWidth=\"720\" maxHeight=\"720\" maxFrameRate=\"30\" par=\"1:1\" lang=\"und\" subsegmentAlignment=\"true\" subsegmentStartsWithSAP=\"1\"\u003e\n   \u003cRepresentation id=\"616287853331251vd\" mimeType=\"video/mp4\" codecs=\"avc1.4D401E\" width=\"720\" height=\"720\" frameRate=\"30\" sar=\"1:1\" startWithSAP=\"1\" bandwidth=\"1392214\" FBQualityClass=\"hd\" FBQualityLabel=\"720w\" FBPlaybackResolutionMos=\"0:100.00,1080:73.75\"\u003e\n    \u003cBaseURL urlExpiration=\"1664060391\"\u003ehttps://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306579382_403169795265464_6841618905692582694_n.mp4?_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026amp;_nc_cat=108\u0026amp;_nc_ohc=6FGTo62l15YAX91OiIU\u0026amp;edm=ALQROFkBAAAA\u0026amp;ccb=7-5\u0026amp;oh=00_AT87BDr4ROMaaQ30wIJsmevReBKDsdOcBLOjabOB72xqhw\u0026amp;oe=632F8BE7\u0026amp;_nc_sid=30a2ef\u003c/BaseURL\u003e\n    \u003cSegmentBase indexRangeExact=\"true\" indexRange=\"840-919\" FBFirstSegmentRange=\"920-353947\" FBSecondSegmentRange=\"353948-637459\" FBPrefetchSegmentRange=\"920-637459\"\u003e\n      \u003cInitialization range=\"0-839\"/\u003e\n    \u003c/SegmentBase\u003e\n   \u003c/Representation\u003e\n  \u003cRepresentation id=\"1110030823239134v\" mimeType=\"video/mp4\" codecs=\"avc1.42C01E\" width=\"480\" height=\"480\" frameRate=\"30\" sar=\"1:1\" startWithSAP=\"1\" bandwidth=\"874806\" FBQualityClass=\"sd\" FBQualityLabel=\"480w\" FBPlaybackResolutionMos=\"0:100.00,1080:54.98\"\u003e\n    \u003cBaseURL urlExpiration=\"1664043146\"\u003ehttps://scontent-ssn1-1.cdninstagram.com/v/t50.2886-16/306594272_1499246513860959_4781320869794637531_n.mp4?_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026amp;_nc_cat=103\u0026amp;_nc_ohc=lClKEJ6vijsAX-jIOJR\u0026amp;tn=wY_QTAydZlMk39Tm\u0026amp;edm=ALQROFkBAAAA\u0026amp;ccb=7-5\u0026amp;oh=00_AT_D3YdNJkMZbI1AWZ9FVu5LdFsbU46sXzWf7Sk9NwTBOw\u0026amp;oe=632F488A\u0026amp;_nc_sid=30a2ef\u003c/BaseURL\u003e\n    \u003cSegmentBase indexRangeExact=\"true\" indexRange=\"838-917\" FBFirstSegmentRange=\"918-234459\" FBSecondSegmentRange=\"234460-438545\" FBPrefetchSegmentRange=\"918-438545\"\u003e\n      \u003cInitialization range=\"0-837\"/\u003e\n    \u003c/SegmentBase\u003e\n   \u003c/Representation\u003e\n  \u003c/AdaptationSet\u003e\n \u003c/Period\u003e\n\u003c/MPD\u003e","video_codec":"avc1.4D401E","number_of_qualities":2,"pk":"2925696872427300223","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"},{"id":"2925701857403507416_3273209062","media_type":1,"image_versions2":{"candidates":[{"width":1440,"height":1440,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT9zEm_w5gf1mMZbG7a33MtDcyPT46SLHu2MluVU3LY1Ww\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT-en5U_w3g9LboJ3iFwz9mK0pBmYIuIWVQlbXLmH68gOA\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT_1qyLi3m7Q2sFro9XyMH9ik_LuXKgwunN_wO_t-xcZXw\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT-ulA6GCWlaYgXVK4Bp3QLY9F9S6-qHnOLwa36MUc4-gQ\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT_UeSVHP-WLCsWm5QkHxzzibr1LPqPHZSAGpTk3UYRmyA\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT9-N8fw6HDmfvSrMtY5n3A-xOwlqt67x0qHgXIS-t7xJQ\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT9bjvP4WX0Q2CS8eSjEiLMuBAs1p_etwoY-86hjboDmSg\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":1080,"height":1080,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s1080x1080\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT-en5U_w3g9LboJ3iFwz9mK0pBmYIuIWVQlbXLmH68gOA\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":750,"height":750,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s750x750_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT_1qyLi3m7Q2sFro9XyMH9ik_LuXKgwunN_wO_t-xcZXw\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":640,"height":640,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT-ulA6GCWlaYgXVK4Bp3QLY9F9S6-qHnOLwa36MUc4-gQ\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":480,"height":480,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s480x480\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT_UeSVHP-WLCsWm5QkHxzzibr1LPqPHZSAGpTk3UYRmyA\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":320,"height":320,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s320x320\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT9-N8fw6HDmfvSrMtY5n3A-xOwlqt67x0qHgXIS-t7xJQ\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":240,"height":240,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s240x240\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT9bjvP4WX0Q2CS8eSjEiLMuBAs1p_etwoY-86hjboDmSg\u0026oe=6334D85A\u0026_nc_sid=30a2ef"},{"width":150,"height":150,"url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/306376890_124775440324780_4797809504527354966_n.jpg?stp=dst-jpg_e35_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=104\u0026_nc_ohc=ad-eIKjG3AcAX-UYoHL\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026ig_cache_key=MjkyNTcwMTg1NzQwMzUwNzQxNg%3D%3D.2-ccb7-5\u0026oh=00_AT_avCksKdcRU4l7uVmR1sU4zVqNAV_TgxrpiFo1zt9c1g\u0026oe=6334D85A\u0026_nc_sid=30a2ef"}]},"original_width":1440,"original_height":1440,"accessibility_caption":"Photo by 언제나, 늘, 항상, 변함없이💛 on September 12, 2022. May be an image of 1 person, sitting and body of water.","pk":"2925701857403507416","carousel_parent_id":"2925701864416431206_3273209062","commerciality_status":"not_commercial"}],"can_see_insights_as_brand":false,"user":{"pk":3273209062,"username":"lucky_seol","full_name":"언제나, 늘, 항상, 변함없이💛","is_private":false,"profile_pic_url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/79936163_583698908866810_2723666885286559744_n.jpg?stp=dst-jpg_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=110\u0026_nc_ohc=YV0_CehBxK0AX-AQndg\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026oh=00_AT912SoSrONKKVthv4ImS2nl7PeIIfDKDAYIi0Vzoq6QMA\u0026oe=63332B1F\u0026_nc_sid=30a2ef","profile_pic_id":"2206457946919959837_3273209062","friendship_status":{"following":false,"outgoing_request":false,"is_bestie":false,"is_restricted":false,"is_feed_favorite":false},"is_verified":false,"has_anonymous_profile_picture":false,"is_unpublished":false,"is_favorite":false,"latest_reel_media":0,"has_highlight_reels":false,"transparency_product_enabled":false,"account_badges":[],"fan_club_info":{"fan_club_id":null,"fan_club_name":null}},"can_viewer_reshare":true,"like_count":13,"has_liked":false,"top_likers":[],"facepile_top_likers":[],"photo_of_you":false,"is_organic_product_tagging_eligible":false,"caption":{"pk":"17946526574239202","user_id":3273209062,"text":".\n.\n.\n장어 먹으러 갔다가,\n둘레길 산책하다가,\n오리배타고싶다는말에\n발 구를자신없어서\n자동차 페달보트 타고 \n선선한바람 느끼며 힐링🌿\n#with할아버지할머니\n#연휴끝\n#내일유치원가는날\n#야호\n#마지막사진은미안🫶","type":1,"created_at":1662991300,"created_at_utc":1662991300,"content_type":"comment","status":"Active","bit_flags":0,"did_report_as_spam":false,"share_enabled":false,"user":{"pk":3273209062,"username":"lucky_seol","full_name":"언제나, 늘, 항상, 변함없이💛","is_private":false,"profile_pic_url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/79936163_583698908866810_2723666885286559744_n.jpg?stp=dst-jpg_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=110\u0026_nc_ohc=YV0_CehBxK0AX-AQndg\u0026tn=wY_QTAydZlMk39Tm\u0026edm=ALQROFkBAAAA\u0026ccb=7-5\u0026oh=00_AT912SoSrONKKVthv4ImS2nl7PeIIfDKDAYIi0Vzoq6QMA\u0026oe=63332B1F\u0026_nc_sid=30a2ef","profile_pic_id":"2206457946919959837_3273209062","is_verified":false},"is_covered":false,"is_ranked_comment":false,"media_id":"2925701864416431206","private_reply_status":0},"featured_products_cta":null,"comment_inform_treatment":{"should_have_inform_treatment":false,"text":"","url":null,"action_type":null},"sharing_friction_info":{"should_have_sharing_friction":false,"bloks_app_url":null,"sharing_friction_payload":null},"can_viewer_save":true,"is_in_profile_grid":false,"profile_grid_control_enabled":false,"organic_tracking_token":"eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dGljc190cmFja2VkIjpmYWxzZSwidXVpZCI6IjY1MjA5YWFhNWE4MzRmYWI5Mjc4MjFiYmI1MGU5ZmZjMjkyNTcwMTg2NDQxNjQzMTIwNiIsInNlcnZlcl90b2tlbiI6IjE2NjM4OTMyMzUwODR8MjkyNTcwMTg2NDQxNjQzMTIwNnw1NTY5MDU0NjEyNnwwMDFhMWJmYWEwZDRhMmJkM2U3MDEyNjFlYWI4OTQyMGNjY2I0YzQ0YWEwOTc5MjhkYjlkZGZkZWU5ZDE4ZjIwIn0sInNpZ25hdHVyZSI6IiJ9","has_shared_to_fb":0,"product_type":"carousel_container","deleted_reason":0,"integrity_review_decision":"pending","commerce_integrity_review_decision":null,"music_metadata":{"music_canonical_id":"0","audio_type":null,"music_info":null,"original_sound_info":null,"pinned_media_ids":null},"is_artist_pick":false}],"num_results":1,"more_available":false,"auto_load_more_enabled":false,"status":"ok"}

In [11]:
keyword = '17년생아들'
tag_ex_url = f'https://www.instagram.com/explore/tags/{keyword}/'

In [12]:
browser.get(tag_ex_url)

In [27]:
ID_SET = set()

In [50]:
# 무한 스크롤(유휴 시간 적당히 조절 필요)
body = browser.find_element(by=By.TAG_NAME, value='body')
for i in range(500):
    body.send_keys(Keys.PAGE_DOWN)
    sleep(2)

In [55]:
rownum = 0
# Access requests via the `requests` attribute
for request in browser.requests:
    if request.response:
        if 'sections' in request.url:
            body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
            data = json.loads(body)
            
            sections = data['sections']
            for section in sections:
                medias = section['layout_content']['medias']
                for media in medias:
                    rownum += 1                    
                    if rownum % 1000 == 0:
                        print(rownum, '번 째 완료')
                        
                    username = media['media']['user']['username']
                    if username not in ID_SET:
                        ID_SET.add(username)  

1000 번 째 완료
2000 번 째 완료
3000 번 째 완료
4000 번 째 완료
5000 번 째 완료


In [54]:
len(ID_SET)

515

In [56]:
ADDED_SET = set()

In [66]:
target_set = ID_SET - ADDED_SET
len(target_set)

65

In [67]:
target_set

{'86.89.16___',
 '90____ming',
 '97_08_12_jk',
 '__na.rae__',
 '_s_j_i_k',
 'a_yang_mom',
 'anak_ssuu',
 'bboing815',
 'bitnanila',
 'chaevely_mommy',
 'chlwls5',
 'coeeunjeong0',
 'dani_damin',
 'doo_mom2',
 'dukzilll_3',
 'eunhee_suhyo12',
 'fanjungeun',
 'ggaeggeumbal',
 'ggom_sun',
 'ghrthrtyjh67',
 'gokoro8570',
 'ha16.you18',
 'hada_sisters',
 'haha_nammae',
 'hankyum_j',
 'hthrth565',
 'htrjutyk78',
 'hye__b.rin',
 'hyeran6658',
 'jaemin.lee',
 'jamjam__2311',
 'jangsyonggg',
 'jiaplus__',
 'jin_ju_mom',
 'jo.jo_1999_',
 'jooa1014',
 'joujou_atelier',
 'ju_lia_k__',
 'k_j_gahyeon',
 'kidspoolvilla_otete',
 'laboom160218',
 'lappuni001',
 'lovechoi29',
 'momozzi0802',
 'my_sweetie_reah',
 'rghrthtrh67',
 'sdylove35',
 'seowoo__lune',
 'seulla_36',
 'sh_870303',
 'simple_bapsang',
 'sssssun._.o',
 'sugarhello',
 'tia_chhhh',
 'timehuk0517',
 'wjdalrnd_',
 'wonji96',
 'wow_ddaann_bbii',
 'yepp_92',
 'yolobin.v',
 'yr___sss',
 'yuras_0109',
 'yurayudongmom',
 'zion__mama',
 'zzang_i

In [70]:
target_set = ID_SET - ADDED_SET
rownum = 0
for user_id in target_set:
    rownum += 1
    data = get_user_by_user_id(user_id, headers)
    fullname = data['user']['full_name']
    follower_cnt = data['user']['edge_followed_by']['count']
    following_cnt = data['user']['edge_follow']['count']
    print(rownum, user_id, fullname, follower_cnt, following_cnt)
    write_excel([rownum, user_id, unicodedata.normalize('NFC',fullname), follower_cnt, following_cnt])
    ADDED_SET.add(user_id)
    
    if rownum % 90 == 0:
        sleep(600)
        print("10분간 휴식")
    else:
        sleep(3)

1 jooa1014 곰돌담요챠챠 10628 1946
2 jin_ju_mom kyungim 151 71
3 yurayudongmom 유라동스맘 19 7
4 k_j_gahyeon 정민혜 92 150
5 bitnanila 연년생 예나맘❤# 곰손엄마 간단미술놀이👩‍🎨 4987 1032
6 86.89.16___  524 322
7 seulla_36 짜니뿡 1337 1670
8 yepp_92 서정민 222 263
9 fanjungeun 인☆ 173 93
10 sugarhello sugar 909 370
11 97_08_12_jk 진경 836 207
12 ggaeggeumbal 공동육아 깨끔발어린이집 401 0
13 hada_sisters 하다자매 146 142
14 simple_bapsang 심플밥상 258 192
15 __na.rae__ RN.날 187 157
16 wonji96 하원지 570 428
17 zzang_irene jangeunjeong 147 368
18 yuras_0109 유남매그램 1150 1466
19 hankyum_j ᴊ ʜᴀɴ kyum 286 410
20 doo_mom2 JungHye Mun 515 182
21 laboom160218 🌹인천🌹반영구🌹속눈썹연장💞(블리브로우)velybrow 419 265
22 rghrthtrh67 육아맘부업 워킹맘일상 재테크문의 39 0
23 gokoro8570 고코로 690 1550
24 jamjam__2311 잼잼 145 99
25 ghrthrtyjh67 육아맘부업 워킹맘일상 재테크문의 25 0
26 joujou_atelier 주주아뜰리에 24873 1434
27 anak_ssuu 혜민 657 840
28 chlwls5 예빈이맘♡ 673 278
29 eunhee_suhyo12 다둥이네👉👨‍👦‍👦👩‍👧‍👧👈 505 680
30 _s_j_i_k  9 82
31 sssssun._.o 영선이🙈 927 688
32 hyeran6658 이혜란 473 446
33 coeeunjeong0 연년생

## 공통 함수

In [53]:
def get_short_code(target_url):
    short_code = ''
    try:
        short_code = target_url.split("/")[4]
    except Exception as e:
        short_code = ''
    finally:
        return short_code

In [54]:
def get_datetime(unixtimestamp):
    return datetime.datetime.utcfromtimestamp(int(unixtimestamp)).strftime('%Y-%m-%d %H:%M:%S')

In [55]:
def get_app_id(res_text):
    prog = re.compile('"\d{15}"')
    result = prog.findall(res_text)
    return result[0].replace('"','')

In [56]:
import requests
from io import BytesIO
from PIL import Image

def download_img(url, filename):
    res = requests.get(url)
    img = Image.open(BytesIO(res.content))
    img.save('./output/'+ filename, 'JPEG')

## 대상 정보 세팅

In [12]:
target_url = 'https://www.instagram.com/p/Cg-57NAAI05/'
# target_url = 'https://www.instagram.com/p/ChZZuolPKBo/'

short_code = get_short_code(target_url)
short_code

'Cg-57NAAI05'

## 특정 게시글의 좋아요 추출

In [13]:
def get_likers(browser, short_code):

    end_cursor = ''
    variables = {}
    variables['shortcode'] = short_code
    variables['first'] = 50
    variables['after'] = end_cursor
    has_next_page = True
    rownum = 0

    while (has_next_page):

        variables['after'] = end_cursor
        json_variables = str(json.dumps(variables))

        url = f'{LIKER_URL}&variables={json_variables}'

        browser.get(url)
        sleep(3)        
        content = browser.find_element(by=By.TAG_NAME, value='pre').text
        data = json.loads(content)

        has_next_page = data['data']['shortcode_media']['edge_liked_by']['page_info']['has_next_page']
        end_cursor = data['data']['shortcode_media']['edge_liked_by']['page_info']['end_cursor']
        likers = data['data']['shortcode_media']['edge_liked_by']['edges']

        for liker in likers:
            rownum += 1
            write_excel('likers', [rownum, liker['node']['username'], unicodedata.normalize('NFC',liker['node']['full_name'])])
          # print(rownum, liker['node']['username'], liker['node']['full_name'])

In [14]:
get_likers(browser, short_code)

## 특정 게시글의 댓글 추출

In [92]:
'''
{"node":
    {"id":"17939707148474198",
     "text":"🙆‍♀️정답 봄나들거리🙆‍♀️#춘천원도심르네상스사업단 팔로우완료 😄리그램완료 💟💟 항상 응원합니다 🔥  @pioniaflower @h.he0k @shm3259.in @lovely_suyeon21 @skylake_travel @sytwinkle14",
     "created_at":1660698082,
     "did_report_as_spam":false,
     "owner":
         {"id":"4543352426",
          "is_verified":false,
          "profile_pic_url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/289831110_144259568201132_6421349397175006169_n.jpg?stp=dst-jpg_s150x150\u0026_nc_ht=scontent-ssn1-1.cdninstagram.com\u0026_nc_cat=108\u0026_nc_ohc=jArKHNM9MRMAX9RD5Jn\u0026tn=ue7Jb2fRs_DBLTiV\u0026edm=AMrZAGUBAAAA\u0026ccb=7-5\u0026oh=00_AT_0-Q3aqgkPmbmubk75JsG9mygOFd4wKADg56JdMFaHzw\u0026oe=63084F7A\u0026_nc_sid=9fb510",
          "username":"skylake_travel"},
     "viewer_has_liked":false,
     "edge_liked_by":{"count":0},
     "is_restricted_pending":false,
     "edge_threaded_comments":
         {"count":0,
          "page_info":
              {"has_next_page":false,
               "end_cursor":null},
           "edges":[]}}},
'''

'\n{"node":\n    {"id":"17939707148474198",\n     "text":"🙆\u200d♀️정답 봄나들거리🙆\u200d♀️#춘천원도심르네상스사업단 팔로우완료 😄리그램완료 💟💟 항상 응원합니다 🔥  @pioniaflower @h.he0k @shm3259.in @lovely_suyeon21 @skylake_travel @sytwinkle14",\n     "created_at":1660698082,\n     "did_report_as_spam":false,\n     "owner":\n         {"id":"4543352426",\n          "is_verified":false,\n          "profile_pic_url":"https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/289831110_144259568201132_6421349397175006169_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=jArKHNM9MRMAX9RD5Jn&tn=ue7Jb2fRs_DBLTiV&edm=AMrZAGUBAAAA&ccb=7-5&oh=00_AT_0-Q3aqgkPmbmubk75JsG9mygOFd4wKADg56JdMFaHzw&oe=63084F7A&_nc_sid=9fb510",\n          "username":"skylake_travel"},\n     "viewer_has_liked":false,\n     "edge_liked_by":{"count":0},\n     "is_restricted_pending":false,\n     "edge_threaded_comments":\n         {"count":0,\n          "page_info":\n              {"has_next_page":false,\n               "end_cu

In [93]:
def get_comments(browser, short_code):

    end_cursor = ''
    variables = {}
    variables['shortcode'] = short_code
    variables['first'] = 50
    variables['after'] = end_cursor
    has_next_page = True
    rownum = 0

    while (has_next_page):
        variables['after'] = end_cursor
        json_variables = str(json.dumps(variables))

        url = f'{COMMENT_URL}&variables={json_variables}'

        browser.get(url)
        sleep(3)
        
        content = browser.find_element(by=By.TAG_NAME, value='pre').text
        data = json.loads(content)

        has_next_page = data['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['has_next_page']
        end_cursor = data['data']['shortcode_media']['edge_media_to_parent_comment']['page_info']['end_cursor']
        comments = data['data']['shortcode_media']['edge_media_to_parent_comment']['edges']

        for comment in comments:
            rownum += 1
            write_excel('comments', [rownum, 1, comment['node']['owner']['username'], comment['node']['text']])
              # print(rownum, 1, comment['node']['owner']['username'], comment['node']['text'])
            co_comments = comment['node']['edge_threaded_comments']['edges']
            for co_comment in co_comments:
                write_excel('comments', [rownum, 2, co_comment['node']['owner']['username'], co_comment['node']['text']])
                # print(rownum, 2, co_comment['node']['owner']['username'], co_comment['node']['text'])
            
            if rownum % 100 == 0:
                print(f'댓글 {rownum}개 추출 완료')

In [94]:
get_comments(browser, short_code)

## 댓글 수동으로 추출(왜 인지는 모르겠으나, graphQL로 추출 안 되는 경우)

In [15]:
def get_comments_v1():
    # headers 값은 변경 필요
    media_no = '2900009967132118329'
    headers = {
        "cookie": 'mid=YwL1ggAEAAHv2HpDOIcH7MuBG2OF; ig_did=B0DE67DD-AF01-4EE2-B4E5-8BDE50356FB4; ig_nrcb=1; csrftoken=7emj7gEwa8jVTq4eo8OzljmyUSjySJZw; ds_user_id=7429511270; sessionid=7429511270%3AAsxXfYcdyn2bXr%3A8%3AAYcJTCf2Fd4qvUqe38zr2Z9UBD0aBM5MXC5TjCOYxQ; shbid="4632\0547429511270\0541692674316:01f73348eaba54882f05b8c50683a65857cd93bf7fd3b8ea4313d6b7723a3df65ec56dd8"; shbts="1661138316\0547429511270\0541692674316:01f7a67aa7a6e0ffce0eb8bc2521fdfdf20c8a4039825163e12aefd68ad245d73b4550e6"; dpr=2; datr=v_UCY5eq_Ej8ngdhLk1beD-T; rur="NAO\0547429511270\0541692674389:01f7c21bf400a98ba44b73cce981ea91d187bd8a6f3e16081f0596fd8fa4f70ae2a1e19f"'
        , "referer" : "https://www.instagram.com/"
        , "x-asbd-id": "198387"
        , "x-ig-app-id": "936619743392459"
        , "x-csrftoken": "7emj7gEwa8jVTq4eo8OzljmyUSjySJZw"
        , "x-ig-www-claim": "hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3Kpfv"
    }
    init_url = f'https://i.instagram.com/api/v1/media/{media_no}/comments/?can_support_threading=true&permalink_enabled=false'
    
    has_next = True
    rownum = 0
    min_id = ''    
    
    while(has_next):
        if len(min_id) > 0 :
            req_url = f'https://i.instagram.com/api/v1/media/{media_no}/comments/?can_support_threading=true&min_id={min_id}'
        else:
            req_url = init_url
        
        res = requests.get(req_url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        data = json.loads(res.text)

        for comment in data['comments']:
            rownum += 1
#             print(rownum, 1, comment['user']['username'],comment['text'], get_datetime(comment['created_at']))
            write_excel('comments', [rownum, 1, comment['user']['username'], comment['text'], get_datetime(comment['created_at'])])

            # 대댓글 추출
            if comment['child_comment_count'] > 0:
                comment_id = comment['pk']
                child_comment_url = f'https://i.instagram.com/api/v1/media/{media_no}/comments/{comment_id}/child_comments/?max_id='
                child_res = requests.get(child_comment_url, headers=headers)
                child_data = json.loads(child_res.text)
                
                for child in child_data['child_comments']:
                    write_excel('comments', [rownum, 2, child['user']['username'], child['text'], get_datetime(child['created_at'])])

    
        if 'next_min_id' in data:
            min_id = data['next_min_id']
        else:
            has_next = False
        
        if rownum % 100 == 0:
            print(f'{rownum}번째 추출 완료')

In [16]:
get_comments_v1()

## 팔로워 추출 v1

In [30]:
def get_follower_v1():
    # headers 값은 변경 필요
    user_id = '572332018'
    init_url = f'https://i.instagram.com/api/v1/friendships/{user_id}/followers/?count=12&search_surface=follow_list_page'
    headers = {
        "cookie": 'mid=YyPu8AAEAAFbctYJPtjkeV47pA36; ig_did=CE36208A-9232-41D7-9C78-FCE368F9CB9B; ig_nrcb=1; csrftoken=eqlRiRpZrYxrw7D4EkkrLnpnmPhxlltw; ds_user_id=7429511270; sessionid=7429511270%3AoaRqfkjS6LU4Nq%3A19%3AAYdVG3P3mbLHBvZocHV7PThdM7zqnCrkkjvqBrx8TQ; shbid="4632\0547429511270\0541694835323:01f74e6ad7ea00675b76324d7b03eb658ed03de81a5c4bd7539a5b0b57c8c9e6e41be026"; shbts="1663299323\0547429511270\0541694835323:01f7dd472b49d973d251dd5a2ff9a68abb8b901aa0d046505447e6f06733a310b444069e"; rur="NAO\0547429511270\0541694835383:01f7f6c447aa59447d6e0253b2eb9eed12b92f79fd1bcc3defb9dbf098899d82cd666cee"'
        , "referer" : "https://www.instagram.com/"
        , "x-asbd-id": "198387"
        , "x-ig-app-id": "936619743392459"
        , "x-csrftoken": "eqlRiRpZrYxrw7D4EkkrLnpnmPhxlltw"
        , "x-ig-www-claim": "hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3Kg77"
    }
    
    has_next = True
    rownum = 0
    max_id = 'QVFEN2xJN0syY1hocWRWVlVISVFnSDdsSEVIUHIwWXVRNXJXbHpyeG9xSGhWYUlZN3RkQTZmOGtNVEZEYS1rYkpjM3BmWHUzVjFpZkdFbGw5ZHM0T2drMA=='
    
    while(has_next):
        if len(max_id) > 0 :
            req_url = f'https://i.instagram.com/api/v1/friendships/{user_id}/followers/?count=12&max_id={max_id}&search_surface=follow_list_page'
        else:
            req_url = init_url
        
        res = requests.get(req_url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        data = json.loads(res.text)
        for user in data['users']:
            rownum += 1
#             print(rownum, user['username'],user['full_name'])
        
        if 'next_max_id' in data:
            max_id = data['next_max_id']
            print(max_id)
        else:
            has_next = False
        
        if rownum > 10:
            break

In [31]:
get_follower_v1()

{'pk': 50728279229, 'username': 'caroline.eyes', 'full_name': '欢迎光临', 'is_private': False, 'profile_pic_url': 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/269863347_203622141890471_4300561697895184766_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=3d0Y9nCUFv8AX8FUMV9&edm=APQMUHMBAAAA&ccb=7-5&oh=00_AT-wnw1ZqU69aWaif_PQE3NQJToBDUv8bnq9estBIyHiMw&oe=632AFB78&_nc_sid=e5d0a6', 'profile_pic_id': '2735905512896717666_50728279229', 'is_verified': False, 'has_anonymous_profile_picture': False, 'has_highlight_reels': False, 'account_badges': [], 'similar_user_id': None, 'latest_reel_media': 0}
{'pk': 48602933525, 'username': 'heyday_hh43', 'full_name': '', 'is_private': True, 'profile_pic_url': 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-19/296364003_801719190838034_1126040587229152533_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=s3Bw5tG1tdIAX8dJiMj&tn=jyhcDCRtMguxxV43&edm=APQMUHMBAAAA&ccb=7-5&o

## 팔로워 추출 v2 

In [58]:
def get_follower_v2():
    graphql_endpoint = 'https://www.instagram.com/graphql/query/?query_hash=37479f2b8209594dde7facb0d904896a&variables={}'

    # ID 변경 필요
    variables = {}
    variables['id'] = '572332018'
    variables['first'] = 50
#     variables['after'] = 'QVFBYUtOWWZLYWlUZE53UHB4NmVJdlVsUWlySHZsakZTcjMzTWtzNnVCUHBzaFhlekRqRHNNbWFRNV9hZjJWRkVUSkozaXNCcDh0eFZxQnFEODlSeS1YVQ=='
    
    sc_rolled=0
    has_next_data = True
    rownum = 0
    
    url = graphql_endpoint.format(str(json.dumps(variables)))
    sleep(2)
    browser.get(url)
    
    while has_next_data:
        pre = browser.find_element(by=By.TAG_NAME, value="pre").text
        data = json.loads(pre)['data']

        # get followers
        page_info = (data['user']['edge_followed_by']['page_info'])
        edges = data['user']['edge_followed_by']['edges']
        for user in edges:
            try:
                rownum += 1
                write_excel([rownum, user['node']['username'], user['node']['full_name'].encode('utf-8')])
            except Exception as e:
                pass

        
        has_next_data = page_info['has_next_page']
        if has_next_data:
            variables['after'] = page_info['end_cursor']
            print(variables['after'])
            url = graphql_endpoint.format(str(json.dumps(variables)))
#             sleep(2)
            browser.get(url)
            sc_rolled+=1

            if sc_rolled > 91:
                print("10분간 휴식. 현재 건 수 : ", rownum)
                break
                sleep(600)
                sc_rolled = 0

        
#         if rownum >100:
#             break

In [59]:
get_follower_v2()

after :  QVFERkxPUm9IT3Z0UTBsNUx1UnBWbmF2dmgxODNzX05QcVhjTUNSb2o5WlpDTG1FaW55eVByM0hSc0E5TG5zWEtsY09MUGlTQUNiQV9CcGlTem9RMXVPVA==
after :  QVFEOWFYakN0a1VaVlBZU3VQZENsVkk1amhfTnFEZFdVeG1ST2Y5Mmx3andjMjNRUlk5Smx4LWxfal9FVk1Bd2h5S0FncE0zNmtYSFRib3VUODM5amFEbw==
after :  QVFDYldxZ2JhSnBkZUtCTU9BZm5ZQ1UzLV9XMWs5a3pHTjN6TWZjallLZnRhZXhlYlZURlV6UDRZOXhCOFgxNkRKSEh6NlJKSm93MldNQUVDcmpGWmVZZQ==
after :  QVFENnZTd1o3akJQRUN0SVVyYXh3dXBwMG4yMFpPRkJsYTBlSWZTd2RVWXN0N1VJbWpEZjJHVG5ZQVZyVDk4Uk1zSVZCS2VhVE00Q2hSYW1rM1JQb2ZycA==
after :  QVFBZG15eDI3REVFaDk0Z1VWdmxQLVFidTl4M3lUbDcxRlVDb1owTks3aGRScVY2RzY2UHM4X3pFRHdsa1lkQ3VfUGJHZVpSU25hd3NuWnE3Wk9pMGVqZA==
after :  QVFCU2VmMUxHajBrWVY4aXhlRXF5WExadUl2SmQxNklxdHFhZndmRmd3eEV1emM4eUxud2xqc1dxR2szbjJIeDlzdWQ5T3V3MlZpbWlnei1MRmoxd3djZw==
after :  QVFCUzVwMzhfV3VKRks1NUw1N2lPOFRjNzJwOXpMVGh1a1ZzcTlnZU8tenM4N1NxRXpZM056TzFCbWJZQXZYaGZzR2IwNF9rNUV3cHNyY1B2U2pjUDlqQQ==
after :  QVFBSmdQdFhhV1FwM1lwUzE5OE9wNm1uRlNGLTFoRUk0TmpJMmRJOGNpRExOTG9OVW1OYk1BQzZrQkF2Z

after :  QVFEQXpvUFlwcnpzRTRVQTJBY2pFOURMbkFoNHJkeFVBT0tSbkM1VlNfaVVySDVYbHZsekVWQWVWMDVHS25jZ29RZWNmWHE1eWVNTTFqZ2xieTVZbjF4Tg==
after :  QVFCTW1pQkZEREpzeHIwSTMxb2xadmgyYjcxUXRoNGNrUFN0QVVNUmdZRG9lUUUtdEdtODBNcGN4MTRkazVEaHVQTTVvZjQweFZaa0hESVBEbFlGVnlUVQ==
after :  QVFEY3RJU1JBaXgybExHeTVlMzdGSGswclRxOU1zYVhLMVVxSGNhUUM3SFl0d2tRMXZGMF9LSUxYM1BSSEtHdlpfVFF3RmN2NUZaMEZETjBaa2pxaUZhaQ==
after :  QVFCSTNfbE9LUTNEMEZXeFdUNUdIYXpsZXFDZ0FFYmt0bF9pa2JjaFRfZHVxR2M1TzQtb0tjRm5pcmh2Zk1IaDhmck1HMDZFX3RpbEpBTXhSRURyblFyeQ==
after :  QVFEMXFkWklkcjdCVGRMekctMEpFcEo3RUJ6V1dYTDhPRnMxY2xoVUUyU1lmTkRQYzdKcDZXRXQwbGdpVmhOdlVUTmJ4TWZQM1U0bldiYTRzWEJQd1VDbA==
after :  QVFCczhpdWgyYzhva1ZoclJYTlVzc2ZSaDRGZFQ4NmtrZkdONFFUd1I1R1lrRFBRQWxWaG0xNEZWRHhyQmpXbHkwSkh2N2dUZm4wckRNeElaTF93QUxjUQ==
after :  QVFEbzNMQlNEVUVoMFZNS2VVQ2NoMDQxaWpEQmtiQXU2YzJBbndxUkdtdjQ1dWlITjBaa1c3aUJXZEQxY0hNWnREcExFRzVJMmpmMnlieDVxNTRVWlJGWA==
after :  QVFBajlSTnRVMm41UU53ZV9DeFozbmhJSkl3U1pQVndFSTh2NGVNRnR5U0dRR2NQcFZhN0pWRElVcGNBN

IllegalCharacterError: 

## 사용자 정보 수집

In [12]:
headers = {
        "cookie": 'mid=YxlvXAAEAAFCY4VaqJl5S2tyTlha; ig_did=C42E8AB6-5505-451C-865B-9DC095EC4338; ig_nrcb=1; csrftoken=ZEywrSWpx3UhV3UlfuSZiOoCT1WZHNzn; ds_user_id=7429511270; sessionid=7429511270%3AqVvDXpyCbjao8S%3A22%3AAYfXKsBcPu2kCrqC8AJKkqj-a8kT5HVoeiepO_8BbA; shbid="4632\0547429511270\0541694147304:01f7807b4a64f9a5bdd609eae6eac2dd08b6926fd6d368d12f0676c7362de3ba6a036d05"; shbts="1662611304\0547429511270\0541694147304:01f7d65fcd7f3084deb080eef32bf78f07c2ec9bbbf239a72ae6798024b4740ca0c69803"; dpr=2; datr=fG8ZY6A-QhF2MLpJ9qHhaLZn; rur="NAO\0547429511270\0541694147455:01f78d1835f9f7bc8665e7285b632b0659142caa741b930e56bcf0380a36602e733676a3"'
        , "referer" : "https://www.instagram.com/"
        , "x-asbd-id": "198387"
        , "x-ig-app-id": "936619743392459"
        , "x-csrftoken": "jbV6j70FpOV7OPcFXlQeFcPCyO1AmKWr"
        , "x-ig-www-claim": "hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3KpiA"
    }

In [13]:
user_id = 'willbesnjob'

In [14]:
data = get_user_by_user_id(user_id, headers)
fullname = data['user']['full_name']
follower_cnt = data['user']['edge_followed_by']['count']
following_cnt = data['user']['edge_follow']['count']

## 사용자 타임라인 게시글 추출

In [ ]:
'https://www.instagram.com/graphql/query/?query_hash=69cba40317214236af40e7efa697781d&variables=%7B%22id%22%3A%2231710018795%22%2C%22first%22%3A12%2C%22after%22%3A%22QVFBZVJ3Q28zbHV2TEM4Z0QxMjJJcEZHUXBVNkdOZHFoZUJ3dFZIbGlsMm84WXlIMHZrb1BmLVd4NzU4Yzl0aXVnUGZsZFp5eG1SVWZsZjF0dUh5cEVQMQ%3D%3D%22%7D'

In [24]:
data['user']['edge_owner_to_timeline_media']['edges'][0]

{'node': {'__typename': 'GraphImage',
  'id': '2917430207541748582',
  'shortcode': 'Ch8y1EJhSNm',
  'dimensions': {'height': 1080, 'width': 1080},
  'display_url': 'https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/302176830_415539767349072_5499940133874077676_n.webp?stp=dst-jpg_e35&_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_cat=100&_nc_ohc=OJ4jG-wA0MkAX8aThcd&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AT-rL80Wizz75pZcmqYjoJOWtD5cNEVbGuIg6ScZGA-h0A&oe=631F9C8E&_nc_sid=8fd12b',
  'edge_media_to_tagged_user': {'edges': []},
  'fact_check_overall_rating': None,
  'fact_check_information': None,
  'gating_info': None,
  'sharing_friction_info': {'should_have_sharing_friction': False,
   'bloks_app_url': None},
  'media_overlay_info': None,
  'media_preview': 'ACoq1aQ5qeRlboOaoSSo/wAue457fTI9/wDPNeY462Wvmjpv8ifJ9KASe1UWADMMngen9c9u/wD9enx3Kqe56dv8/wCfenyhc0DGQMmmUgk8wbucH1pal2vp+I15kd0ZUx5S7snn+n5+vaqrLIWPyjHTlTyM+2f6ZrYIzVGSxDksWYZ54x/h0rpcOX07/wCZle5SPmbQ20FicH5ewGR6dDSrE2Sdoz04BHfjr24HTt1NaAg

## 태그로 타임라인 추출

In [69]:
def get_timeline_by_tag(keyword):

    tag_ex_url = f'https://www.instagram.com/explore/tags/{keyword}/'
    tag_url = f'https://i.instagram.com/api/v1/tags/web_info/?tag_name={keyword}'
    section_url = f'https://i.instagram.com/api/v1/tags/{keyword}/sections/'

    # 최초 조회
    browser.get(tag_ex_url)
    res = browser.page_source
    app_id = get_app_id(res)
    csrftoken = browser.get_cookie("csrftoken")['value']
    
    cookies = browser.get_cookies()
    header_cookie = ''
    for cookie in cookies:
        header_cookie += f"{cookie['name']}={cookie['value']}; "
    
    # 헤더 세팅
    headers = {}
    headers['referer'] = 'https://www.instagram.com/'
    headers['x-csrftoken'] = csrftoken
    headers['cookie'] = header_cookie
    headers['x-ig-app-id'] = app_id
    # headers['x-ig-www-claim'] = 'hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3Kk-D'
    # headers['x-asbd-id'] = '198387'

    # 2차 조회
    res = requests.get(tag_url, headers=headers)
    data = res.json()
    sections = data['data']['recent']['sections']
    
    for section in sections:
        medias = section['layout_content']['medias']
        for media in medias:
            username = media['media']['user']['username']
            full_name = media['media']['user']['full_name']
#             print(username, full_name)
    
    
    max_id = data['data']['recent']['next_max_id']
    page = data['data']['recent']['next_page']
    
    # 3차 조회
    payload = f'include_persistent=0&max_id={max_id}&page={page}&surface=grid&tab=recent'

    # 헤더는 변경 필요
#     headers = {
#         "cookie": 'mid=YxbWzwAEAAFBN1fJAsG4DH-okgyK; ig_did=23AF9373-C969-4113-8D10-BEE47AFF5BF9; ig_nrcb=1; csrftoken=jbV6j70FpOV7OPcFXlQeFcPCyO1AmKWr; ds_user_id=7429511270; sessionid=7429511270%3AEfpXupzkXBtkax%3A20%3AAYey4TJ82JvMdhalX_L8WnjmPGN2qAcyke0TBMmU7A; shbid="4632\0547429511270\0541693977178:01f7aed0b2d7ea9550cb530efe4add4ac59c7e248e72ba3dd72320f36b35577388238dbc"; shbts="1662441178\0547429511270\0541693977178:01f7235b9533ddce038722246a1fe36ea12ee80b34cea8049d45d7f80f80c41554c17c97"; rur="NAO\0547429511270\0541693980552:01f700a554677550a1f9e22c7d213320ebed0e040327d1458825e1fd516e60db09913c17"'
#         , "referer" : "https://www.instagram.com/"
#         , "x-asbd-id": "198387"
#         , "x-ig-app-id": "936619743392459"
#         , "x-csrftoken": "jbV6j70FpOV7OPcFXlQeFcPCyO1AmKWr"
#         , "x-ig-www-claim": "hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3KpiA"
#     }
    
    has_next = True
    
    while(has_next):
        print(headers)
        res = requests.post(section_url, headers=headers, data=payload)
        data = res.json()
        
        sections = data['sections']
        for rownum, section in enumerate(sections):
            if rownum == 0:
                pass
            else:
                medias = section['layout_content']['medias']
                for media in medias:
                    username = media['media']['user']['username']
                    full_name = media['media']['user']['full_name']
                    print(username, full_name)
        
        if data['more_available'] is True:
            max_id = data['next_max_id']
#             page = data['next_page']
            page += 1
            payload = f'include_persistent=0&max_id={max_id}&page={page}&surface=grid&tab=recent'
            headers = res.request.headers
            print(payload)
        else:
            has_next = False

In [63]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [70]:
keyword = '17년생아들'
get_timeline_by_tag(keyword)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## 특정 태그로 게시글 올린 계정 정보 추출(seleniumwire로 진행)

In [20]:
from seleniumwire import webdriver as webdriver_wire
from seleniumwire.utils import decode
from selenium.webdriver.common.keys import Keys

In [21]:
browser = webdriver_wire.Chrome(service=Service(ChromeDriverManager().install()))

In [24]:
# 헤더 생성(수작업으로 변경)
headers = {
        "cookie": 'mid=YygccgAEAAF66gUZX89tMvaWQyKe; ig_did=E7DCB49B-6A0B-4447-8DBF-8A27E669CF00; ig_nrcb=1; csrftoken=IpLkRlnXpqTu2NlsaAPO15AqFvCdS0Kd; ds_user_id=7429511270; sessionid=7429511270%3AEQqOUiqZ7o0gi4%3A4%3AAYcU2j5SA2atjztG4U3h-1fEF7uacNuoJ96GLp1p_g; shbid="4632\0547429511270\0541695109120:01f7bc9ffa29872e9a247cf5d07daf302a8232c8b176830995b4b91f1e18eb6a75078d77"; shbts="1663573120\0547429511270\0541695109120:01f7288d6753982bd28b0f4fce90cf47a919a78feaddaf94678d823f8ebb3fd8444cb3dc"; datr=iBwoY7v4tekKauMMkXneWOh_; rur="NAO\0547429511270\0541695109162:01f711df4ac37ca7f21865641b707acf6c5e91faaf8b05b8db640c4f7777d5d7357ac84b"'
        , "referer" : "https://www.instagram.com/"
        , "x-asbd-id": "198387"
        , "x-ig-app-id": "936619743392459"
        , "x-csrftoken": "IpLkRlnXpqTu2NlsaAPO15AqFvCdS0Kd"
        , "x-ig-www-claim": "hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3Kl6u"
        , "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }

In [25]:
keyword = '16년생아들'
tag_ex_url = f'https://www.instagram.com/explore/tags/{keyword}/'

In [26]:
browser.get(tag_ex_url)

In [28]:
ID_SET = set()

In [ ]:
# 무한 스크롤(유휴 시간 적당히 조절 필요)
body = browser.find_element(by=By.TAG_NAME, value='body')
for i in range(500):
    body.send_keys(Keys.PAGE_DOWN)
    sleep(2)

In [33]:
rownum = 0
# Access requests via the `requests` attribute
for request in browser.requests:
    if request.response:
        if 'sections' in request.url:
            body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
            data = json.loads(body)
            
            sections = data['sections']
            for section in sections:
                medias = section['layout_content']['medias']
                for media in medias:
                    rownum += 1                    
                    if rownum % 1000 == 0:
                        print(rownum, '번 째 완료')
                        
                    username = media['media']['user']['username']
                    if username not in ID_SET:
                        ID_SET.add(username)  

1000 번 째 완료
2000 번 째 완료
3000 번 째 완료
4000 번 째 완료
5000 번 째 완료


In [ ]:
len(ID_SET)

In [476]:
rownum = 0
for user_id in ID_SET:
    rownum += 1
    data = get_user_by_user_id(user_id, headers)
    fullname = data['user']['full_name']
    follower_cnt = data['user']['edge_followed_by']['count']
    following_cnt = data['user']['edge_follow']['count']
    print(rownum, user_id, fullname, follower_cnt, following_cnt)
    write_excel([rownum, user_id, fullname.encode('utf-8'), follower_cnt, following_cnt])
    
    if rownum % 100 == 0:
        sleep(300)
    else:
        sleep(1)

1 vinmi486 🙂스마일후야🙂(후맘)일상과 아동의류 및 공구 1548 1042
2 hyun_ji_ss 조현지 305 212
3 ji.____.hyeee 최자 93 362
4 nam0815 키즈모델 김하준 🔗 아기모델 김하담 271 224
5 yeeee_z1 ᰔ7살 찬율이ᰔ 3543 2327
6 rorothy 🌈 책육아하는 랑블리네 #랑랑맘 2212 997
7 parksoohee77 7살 아들키우는 늙은애미😂 1287 391
8 geumgogihouse 금고깃집 260 268
9 a_yang_mom  1674 453
10 hhhh_092 미어캣 같은 삼형제❤️ 2998 1834
11 pickpick0000 #칼맞팔#선팔 환영#여행 1065 715
12 gimjihye591 김지혜 290 215
13 heeejin91 양희진 207 744
14 seonhye.park Seonhye.Park 213 129
15 boyoung_87 Boyoung👯 206 437
16 _chanyul_05.23 ᴄʜᴀɴʏᴜʟ ᴍᴏᴍ 64 29
17 baegong_joo baegongjoo 546 144
18 hey_jane_5 엄마는마술사◡̈ 엄마표놀이 1147 463
19 ann150321 열매맺는 삶👨‍👩‍👧‍👦 1527 478
20 imyerin0110 임예린 563 231
21 kidspoolvilla_otete 광주 전남 담양 오떼떼 키즈풀빌라 2591 13
22 yange_babystory 경기광주 아들둘 워킹맘 육아소통 1112 606
23 m.m.y89 문민영 1006 486
24 songjae.kim 율남매파파 287 248
25 sjj_jo 김하영 [시윤애미] 1930 1088
26 rin_ziii 🐏🐍🐵🐷💓 1052 279
27 lovable_rany 미란쏭🌸 137 77
28 kittyyejin529 𝕪𝕖 𝕛𝕚𝕟 정예진 ❤ 🧡 💛 💚 💙 💜 3268 936
29 yeppeuguna_nail 넌, 오늘도 

251 ej_happy_moments EJ Lee 108 81
252 wani_12.16 해피둥이 #와니형제맘 609 596
253 han_na1109 윤한나 520 283
254 j._.mom 혀니ღ 1254 977
255 un20141012 육아맘부업 21 16
256 songi4123  226 626
257 zialim83 지아 814 759
258 belovskyzhenya  152 1068
259 kaya_helena 훈훈한딸기네 263 135
260 s.w_hanna 🏆최단기최고매출  [𝘁𝗼𝗽𝗰𝗹𝗮𝘀𝘀팀 유한나] 497 450
261 only_ssoya Ssoya 8516 1457
262 bo._.hyun56 bo._.hyun56 276 79
263 1m870907 하율쓰💕 565 152
264 doong_heim 🐯+🐮+🐵 환둥네 5 13
265 ssun7780 차니유니맘🌟 베니넷 855 542
266 ttotto0129 김도영 43 14
267 han_lee.523 Yeong Eun 1998 603
268 luv_horin 호린남매 1967 1840
269 yoom_babies 최유미 654 521
270 heesuni_ 👶현듀니의 일상기록🧒 183 182
271 czkim_meritz 보험사업가 김춘자 1312 1019
272 kwak.j 곽지연 1386 1163
273 elisabeth8511 Ye Eun Elisabeth 287 159
274 go_jihwan 고지화니 580 431
275 song_94.316 안송이 288 345
276 ruffy__baby 루피베이비 2256 3889
277 n.yong1025  143 192
278 ruhamom.life Jeeeun Yun 1260 888
279 sei1108.wj 웅진북클럽 / 김세이팀장💜 122 165
280 young_ji_0755 함영지 28 8
281 ssmm121300 킴선미 ♡ 514 298
282 j00

AttributeError: 'JSONDecodeError' object has no attribute 'message'

In [470]:
'test'.encode('utf-8')

b'test'

In [17]:
# 보령머드축제
# 추출 내역 : 게시물 사진, 본문, 댓글, 해시태그
rownum = 0
sections = ''
# Access requests via the `requests` attribute
for request in browser.requests:
    if request.response:
        if 'sections' in request.url:
            body = decode(request.response.body, request.response.headers.get('Content-Encoding', 'identity'))
            data = json.loads(body)
            
            sections = data['sections']
#             for section in sections:
#                 medias = section['layout_content']['medias']
#                 for media in medias:
#                     rownum += 1                    
#                     if rownum % 1000 == 0:
#                         print(rownum, '번 째 완료')
                        
#                     username = media['media']['user']['username']
#                     if username not in ID_SET:
#                         ID_SET.add(username)  

In [65]:
# 게시글 업로드 타임?
# get_datetime(sections[2]['layout_content']['medias'][0]['media']['taken_at'])

# 게시글 본문
# sections[2]['layout_content']['medias'][0]['media']['caption']['text']

# 댓글
# sections[2]['layout_content']['medias'][0]['media']['comments']

# 게시글 사진
# image_test = sections[0]['layout_content']['medias'][2]['media']['carousel_media'][0]['image_versions2']['candidates'][0]['url']

sections[2]['layout_content']['medias'][0]['media']

{'taken_at': 1662113960,
 'pk': '2918345875631583180',
 'id': '2918345875631583180_53788958092',
 'device_timestamp': 1662113761668007,
 'media_type': 8,
 'code': 'CiADBygvy_M',
 'client_cache_key': 'MjkxODM0NTg3NTYzMTU4MzE4MA==.2',
 'filter_type': 0,
 'is_unified_video': False,
 'should_request_ads': False,
 'caption_is_edited': False,
 'like_and_view_counts_disabled': False,
 'commerciality_status': 'not_commercial',
 'is_paid_partnership': False,
 'is_visual_reply_commenter_notice_enabled': True,
 'original_media_has_visual_reply_media': False,
 'has_delayed_metadata': False,
 'comment_likes_enabled': False,
 'comment_threading_enabled': True,
 'has_more_comments': False,
 'max_num_visible_preview_comments': 2,
 'preview_comments': [{'pk': '17954973686051541',
   'user_id': 5652934886,
   'text': '🙌',
   'type': 0,
   'created_at': 1662116019,
   'created_at_utc': 1662116019,
   'content_type': 'comment',
   'status': 'Active',
   'bit_flags': 0,
   'did_report_as_spam': False,
   '

## 태그 검색 및 게시글 이미지 수집
- 로그인
- tag_ex_url 호출
- app_id 추출
- 

In [12]:
# 헤더 생성(수작업으로 변경)
headers = {
        "cookie": 'mid=Yyz-TwAEAAED4W5rXaXucPDGNhDN; ig_did=86F45643-485A-47BA-BB4A-0DFF50C85C1E; ig_nrcb=1; csrftoken=5iSXxurTt1x3uuWj2RGcs2qBUBJbeTF8; ds_user_id=55690546126; sessionid=55690546126%3AKqQMcsgPhPXTRG%3A28%3AAYcSRz8S6AZJ8yL2HGIayT7TUBYScOqbahwwzguUgQ; rur="EAG\05455690546126\0541695430201:01f7b407aa66a354a6f0420e9486f73e8f3c3ac1938118e76ed4300a7a9db902c88ef290"'
        , "referer" : "https://www.instagram.com/"
        , "x-asbd-id": "198387"
        , "x-ig-app-id": "936619743392459"
        , "x-csrftoken": "5iSXxurTt1x3uuWj2RGcs2qBUBJbeTF8"
        , "x-ig-www-claim": "hmac.AR1ToUQmxO7ELz9SlvFwj4fs2Jmq4X7uZ_UHKMIwzbx6x9Gt"
        , "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    }

In [13]:
keyword = '17년생아들'
tag_url = f'https://i.instagram.com/api/v1/tags/web_info/?tag_name={keyword}'
tag_url

'https://i.instagram.com/api/v1/tags/web_info/?tag_name=17년생아들'

In [14]:
res = requests.get(tag_url, headers=headers)

In [15]:
data = res.json()

In [16]:
data['data']['recent']['next_max_id']

'QVFERVN6YXVjYmNuZko4a0tBYWcxY3BBYWRLbm1DLWtrclhJTFQyeVM3Mm1TdjUzQXRWX09hZV9YWWp2d09vZUJZa01FWTRLcmhxRDFhbFk5U3Z3SzV4MA=='

In [17]:
data['data']['recent']['next_page']

1

In [19]:
data['data'].keys()

dict_keys(['id', 'name', 'media_count', 'follow_status', 'following', 'allow_following', 'allow_muting_story', 'profile_pic_url', 'non_violating', 'related_tags', 'subtitle', 'social_context', 'social_context_profile_links', 'social_context_facepile_users', 'follow_button_text', 'show_follow_drop_down', 'formatted_media_count', 'challenge_id', 'destination_info', 'description', 'debug_info', 'fresh_topic_metadata', 'promo_banner', 'top', 'recent', 'content_advisory', 'warning_message'])

In [20]:
sections = data['data']['recent']['sections']

In [22]:
medias = sections[0]['layout_content']['medias']

In [23]:
medias[0]['media']['user']['username']

'wanso_mommy'

In [35]:
medias[0]['media']

{'taken_at': 1662167942,
 'pk': '2918798711924948745',
 'id': '2918798711924948745_3621958829',
 'device_timestamp': 1662167610577263,
 'media_type': 8,
 'code': 'CiBp_bLJ38J',
 'client_cache_key': 'MjkxODc5ODcxMTkyNDk0ODc0NQ==.2',
 'filter_type': 0,
 'is_unified_video': False,
 'should_request_ads': False,
 'caption_is_edited': True,
 'like_and_view_counts_disabled': False,
 'commerciality_status': 'not_commercial',
 'is_paid_partnership': False,
 'is_visual_reply_commenter_notice_enabled': True,
 'original_media_has_visual_reply_media': False,
 'has_delayed_metadata': False,
 'comment_likes_enabled': False,
 'comment_threading_enabled': True,
 'has_more_comments': True,
 'next_max_id': '18032521777384863',
 'max_num_visible_preview_comments': 2,
 'preview_comments': [{'pk': '17948616947156038',
   'user_id': 6281737124,
   'text': '어머 세상에 귀여워요ㅋㅋ 포즈 까징❤️',
   'type': 0,
   'created_at': 1662183180,
   'created_at_utc': 1662183180,
   'content_type': 'comment',
   'status': 'Active',
 

In [29]:
for section in sections:
    medias = section['layout_content']['medias']
    for media in medias:
#         print(media)
        username = media['media']['user']['username']
        full_name = media['media']['user']['full_name']
        print(username, full_name)

{'media': {'taken_at': 1663669510, 'pk': '2931394774982117479', 'id': '2931394774982117479_1536679620', 'device_timestamp': 1663669188867119, 'media_type': 8, 'code': 'CiuaARDPshn', 'client_cache_key': 'MjkzMTM5NDc3NDk4MjExNzQ3OQ==.2', 'filter_type': 0, 'is_unified_video': False, 'should_request_ads': False, 'caption_is_edited': False, 'like_and_view_counts_disabled': False, 'commerciality_status': 'not_commercial', 'is_paid_partnership': False, 'is_visual_reply_commenter_notice_enabled': True, 'original_media_has_visual_reply_media': False, 'has_delayed_metadata': False, 'comment_likes_enabled': False, 'comment_threading_enabled': True, 'has_more_comments': False, 'max_num_visible_preview_comments': 2, 'preview_comments': [{'pk': '17993013364547849', 'user_id': 39052909351, 'text': '이제 괜찮아?! 너듀 고생했겠네!😢 너듀 건강잘챙기~♡ 보고싶당~~♡', 'type': 0, 'created_at': 1663672640, 'created_at_utc': 1663672640, 'content_type': 'comment', 'status': 'Active', 'bit_flags': 0, 'did_report_as_spam': False, 'shar

In [12]:
csrftoken = res.cookies['csrftoken']
csrftoken

'319GdzT6VGFistrU7Gw8ZSb8YImKp4jl'

In [86]:
res.request.headers

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'ig_did=851DA63D-C6EF-4A46-B090-27F32F1D7433; ig_nrcb=1; mid=YwTYPAAEAAHrxC4D2JU_K_9_Naou; csrftoken=nPzgWTLx24boJjdjoQpOxuUC6riU8f2K'}

In [89]:
res.url

'https://www.instagram.com/accounts/login/?next=/api/v1/tags/web_info/%3Ftag_name%3D%25EC%25A7%2584%25EB%258F%2597%25EA%25B0%259C'

In [28]:
# section_url = f'https://i.instagram.com/api/v1/tags/{keyword}/sections/'
section_url = f'https://i.instagram.com/api/v1/tags/{keyword}/sections/'
section_url

'https://i.instagram.com/api/v1/tags/17년생아들/sections/'

In [32]:
payload = 'include_persistent=0&max_id=&page=&surface=grid&tab=recent'

In [33]:
res = requests.post(section_url, headers=headers, data=payload)

In [34]:
res.request.headers

{'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'cookie': 'mid=Yyz-TwAEAAED4W5rXaXucPDGNhDN; ig_did=86F45643-485A-47BA-BB4A-0DFF50C85C1E; ig_nrcb=1; csrftoken=5iSXxurTt1x3uuWj2RGcs2qBUBJbeTF8; ds_user_id=55690546126; sessionid=55690546126%3AKqQMcsgPhPXTRG%3A28%3AAYcSRz8S6AZJ8yL2HGIayT7TUBYScOqbahwwzguUgQ; rur="EAG,55690546126,1695430201:01f7b407aa66a354a6f0420e9486f73e8f3c3ac1938118e76ed4300a7a9db902c88ef290"', 'referer': 'https://www.instagram.com/', 'x-asbd-id': '198387', 'x-ig-app-id': '936619743392459', 'x-csrftoken': '5iSXxurTt1x3uuWj2RGcs2qBUBJbeTF8', 'x-ig-www-claim': 'hmac.AR1ToUQmxO7ELz9SlvFwj4fs2Jmq4X7uZ_UHKMIwzbx6x9Gt', 'Content-Length': '58'}

In [36]:
data = res.json()
print('next_max_id :', data['next_max_id'])
print('next_page :', data['next_page'])
sections = data['sections']

next_max_id : 7e06eff6cc19450a8e0ab1f393fe9007
next_page : 1


In [46]:
for section in sections:
    if section['layout_type'] == 'media_grid':
        medias = section['layout_content']['medias']
        for media in medias:
            username = media['media']['user']['username']
            full_name = media['media']['user']['full_name']
#             print(media['media']['caption']['text'])
            print(username, full_name)

marryjjung06 고현정
junwoo.diary 준우의 성장일기❣
junwoo.diary 준우의 성장일기❣
shanxi0911 샨시
l702l3 ⋆ ʏ ᴇ ᴏ ᴜ ʟ _ ᴹᴼᴹ
choiarum89 최아름
l702l3 ⋆ ʏ ᴇ ᴏ ᴜ ʟ _ ᴹᴼᴹ
s_miniyuni 삼남매네👦🏻👦🏻👶🏻💕
hamamy80 인생이 쉬워질 때를 기다릴 수 없다!!
junwoo.diary 준우의 성장일기❣
ylangylang_fragrant Hyeri Shin
junwoo.diary 준우의 성장일기❣
haha_nammae 세젤예연년생남매
2nini_mama 니니형제
s.jun1741 곽승준
l702l3 ⋆ ʏ ᴇ ᴏ ᴜ ʟ _ ᴹᴼᴹ
rururoro.mom 우리의소중한성장기록🌿 우리들의시간
hjyr._ 육아계정🧡
shin_sung_han_mom 신이성이한이 이야기
ylangylang_fragrant Hyeri Shin
junwoo.diary 준우의 성장일기❣
junwoo.diary 준우의 성장일기❣
choiarum89 최아름
aaaaaaamom 


In [52]:
## 두번째
payload = 'include_persistent=1&max_id=7e06eff6cc19450a8e0ab1f393fe9007&page=1&surface=grid&tab=recent'

In [53]:
res = requests.post(section_url, headers=headers, data=payload)

In [54]:
res.request.headers

{'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'cookie': 'mid=Yyz-TwAEAAED4W5rXaXucPDGNhDN; ig_did=86F45643-485A-47BA-BB4A-0DFF50C85C1E; ig_nrcb=1; csrftoken=5iSXxurTt1x3uuWj2RGcs2qBUBJbeTF8; ds_user_id=55690546126; sessionid=55690546126%3AKqQMcsgPhPXTRG%3A28%3AAYcSRz8S6AZJ8yL2HGIayT7TUBYScOqbahwwzguUgQ; rur="EAG,55690546126,1695430201:01f7b407aa66a354a6f0420e9486f73e8f3c3ac1938118e76ed4300a7a9db902c88ef290"', 'referer': 'https://www.instagram.com/', 'x-asbd-id': '198387', 'x-ig-app-id': '936619743392459', 'x-csrftoken': '5iSXxurTt1x3uuWj2RGcs2qBUBJbeTF8', 'x-ig-www-claim': 'hmac.AR1ToUQmxO7ELz9SlvFwj4fs2Jmq4X7uZ_UHKMIwzbx6x9Gt', 'Content-Length': '91'}

In [55]:
data = res.json()
print('next_max_id :', data['next_max_id'])
print('next_page :', data['next_page'])
sections = data['sections']

next_max_id : 035c61d96ae849d7a2dcec883f7a5eee
next_page : 1


In [56]:
for section in sections:
    if section['layout_type'] == 'media_grid':
        medias = section['layout_content']['medias']
        for media in medias:
            username = media['media']['user']['username']
            full_name = media['media']['user']['full_name']
#             print(media['media']['caption']['text'])
            print(username, full_name)

marryjjung06 고현정
junwoo.diary 준우의 성장일기❣
junwoo.diary 준우의 성장일기❣
l702l3 ⋆ ʏ ᴇ ᴏ ᴜ ʟ _ ᴹᴼᴹ
shanxi0911 샨시
choiarum89 최아름
l702l3 ⋆ ʏ ᴇ ᴏ ᴜ ʟ _ ᴹᴼᴹ
s_miniyuni 삼남매네👦🏻👦🏻👶🏻💕
hamamy80 인생이 쉬워질 때를 기다릴 수 없다!!
junwoo.diary 준우의 성장일기❣
ylangylang_fragrant Hyeri Shin
junwoo.diary 준우의 성장일기❣
haha_nammae 세젤예연년생남매
2nini_mama 니니형제
s.jun1741 곽승준
l702l3 ⋆ ʏ ᴇ ᴏ ᴜ ʟ _ ᴹᴼᴹ
rururoro.mom 우리의소중한성장기록🌿 우리들의시간
hjyr._ 육아계정🧡
shin_sung_han_mom 신이성이한이 이야기
ylangylang_fragrant Hyeri Shin
junwoo.diary 준우의 성장일기❣
junwoo.diary 준우의 성장일기❣
2nini_mama 니니형제
choiarum89 최아름


In [13]:
tag_ex_url = f'https://www.instagram.com/explore/tags/{keyword}/'
tag_ex_url

'https://www.instagram.com/explore/tags/16년생아들/'

In [14]:
browser.get(tag_ex_url)
res = browser.page_source
app_id = get_app_id(res)

In [17]:
csrftoken = browser.get_cookie("csrftoken")['value']
csrftoken

'jbV6j70FpOV7OPcFXlQeFcPCyO1AmKWr'

In [18]:
cookies = browser.get_cookies()
header_cookie = ''
for cookie in cookies:
    header_cookie += f"{cookie['name']}={cookie['value']}; "

In [20]:
header_cookie

'rur="NAO\\0547429511270\\0541693977322:01f7e848b490edb600e44c51e2269ae3560c1c55b32169379877179dc6f5ce772065c67a"; shbid="4632\\0547429511270\\0541693977178:01f7aed0b2d7ea9550cb530efe4add4ac59c7e248e72ba3dd72320f36b35577388238dbc"; shbts="1662441178\\0547429511270\\0541693977178:01f7235b9533ddce038722246a1fe36ea12ee80b34cea8049d45d7f80f80c41554c17c97"; sessionid=7429511270%3AEfpXupzkXBtkax%3A20%3AAYey4TJ82JvMdhalX_L8WnjmPGN2qAcyke0TBMmU7A; csrftoken=jbV6j70FpOV7OPcFXlQeFcPCyO1AmKWr; ig_nrcb=1; ds_user_id=7429511270; ig_did=23AF9373-C969-4113-8D10-BEE47AFF5BF9; mid=YxbWzwAEAAFBN1fJAsG4DH-okgyK; '

In [21]:
headers = {}
headers['referer'] = 'https://www.instagram.com/'
# headers['x-csrftoken'] = csrftoken
headers['cookie'] = header_cookie
# headers['x-ig-www-claim'] = 'hmac.AR0tgl1y9JYmZ0YUEjTOha63OYPf4xRueei_PJgZjyH3Kk-D'
# headers['x-asbd-id'] = '198387'
headers['x-ig-app-id'] = app_id

res = requests.get(tag_url, headers=headers)

In [22]:
data = res.json()

In [23]:
data.keys()

dict_keys(['count', 'data', 'status'])

In [24]:
data['status']

'ok'

In [129]:
str(data['data']).replace(' ','')[:1000]

"{'id':'17844977512114033','name':'16년생아들','media_count':11491,'follow_status':0,'following':0,'allow_following':1,'allow_muting_story':True,'profile_pic_url':'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/296274060_367153265571110_3665619598438574926_n.jpg?stp=dst-jpg_e15_s640x640&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=JLwlJ9sV10MAX-tmCEg&edm=AGyKU4gBAAAA&ccb=7-5&ig_cache_key=Mjg5MTMxMDc3MTA1NzY2OTMwOQ%3D%3D.2-ccb7-5&oh=00_AT8UQP6SRsg3IJBVHxkQO7sX87Kg3ES2Fz4q2WmX9_VacQ&oe=631DF2E8&_nc_sid=4cb768','non_violating':1,'related_tags':None,'subtitle':'Seeafewtoppostseachweek','social_context':'','social_context_profile_links':[],'social_context_facepile_users':[],'follow_button_text':None,'show_follow_drop_down':False,'formatted_media_count':'11.4K','challenge_id':None,'destination_info':None,'description':None,'debug_info':None,'fresh_topic_metadata':None,'promo_banner':None,'top':{'sections':[{'layout_type':'media_grid','layout_content':{'medias':[{'media':{'

In [146]:
import re

p = re.compile('\'username\':\'[a-zA-Z0-9_]*\'')
result = p.findall(str(data['data']).replace(' ',''))

In [147]:
result

["'username':'sangbum_philip_seo'",
 "'username':'sangbum_philip_seo'",
 "'username':'kittyyejin529'",
 "'username':'dasom_om2020'",
 "'username':'kittyyejin529'",
 "'username':'dasom_om2020'",
 "'username':'hgimim'",
 "'username':'kittyyejin529'",
 "'username':'hgimim'",
 "'username':'kittyyejin529'",
 "'username':'leejunkyun'",
 "'username':'kittyyejin529'",
 "'username':'leejunkyun'",
 "'username':'kittyyejin529'",
 "'username':'kittyyejin529'",
 "'username':'kittyyejin529'",
 "'username':'choiyujeong23'",
 "'username':'jnwings'",
 "'username':'choiyujeong23'",
 "'username':'jnwings'",
 "'username':'choiyujeong23'",
 "'username':'choiyujeong23'",
 "'username':'aileenkim86'",
 "'username':'gimmiya'",
 "'username':'aileenkim86'",
 "'username':'gimmiya'",
 "'username':'gimmiya'",
 "'username':'gimmiya'",
 "'username':'yeongseon_jeong'",
 "'username':'azure_dini'",
 "'username':'yeongseon_jeong'",
 "'username':'azure_dini'",
 "'username':'azure_dini'",
 "'username':'azure_dini'",
 "'use

In [48]:
data['data']['recent'].keys()

dict_keys(['sections', 'more_available', 'next_max_id', 'next_page', 'next_media_ids'])

In [59]:
data['data']['recent']['sections'][0]['layout_content']['medias']

[{'media': {'taken_at': 1662360349,
   'pk': '2920412739585449559',
   'id': '2920412739585449559_44424708138',
   'device_timestamp': 1013267598120604,
   'media_type': 1,
   'code': 'CiHY-nkhB5X',
   'client_cache_key': 'MjkyMDQxMjczOTU4NTQ0OTU1OQ==.2',
   'filter_type': 0,
   'accessibility_caption': 'Photo by @feeling.anny on September 04, 2022. May be an image of food.',
   'is_unified_video': False,
   'should_request_ads': False,
   'caption_is_edited': False,
   'like_and_view_counts_disabled': False,
   'commerciality_status': 'not_commercial',
   'is_paid_partnership': False,
   'is_visual_reply_commenter_notice_enabled': True,
   'original_media_has_visual_reply_media': False,
   'has_delayed_metadata': False,
   'comment_likes_enabled': False,
   'comment_threading_enabled': True,
   'has_more_comments': True,
   'next_max_id': '17943658463259854',
   'max_num_visible_preview_comments': 2,
   'preview_comments': [{'pk': '17854333172815124',
     'user_id': 2010242192,
     

In [148]:
data['data']

{'id': '17844977512114033',
 'name': '16년생아들',
 'media_count': 11491,
 'follow_status': 0,
 'following': 0,
 'allow_following': 1,
 'allow_muting_story': True,
 'profile_pic_url': 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/296274060_367153265571110_3665619598438574926_n.jpg?stp=dst-jpg_e15_s640x640&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=102&_nc_ohc=JLwlJ9sV10MAX-tmCEg&edm=AGyKU4gBAAAA&ccb=7-5&ig_cache_key=Mjg5MTMxMDc3MTA1NzY2OTMwOQ%3D%3D.2-ccb7-5&oh=00_AT8UQP6SRsg3IJBVHxkQO7sX87Kg3ES2Fz4q2WmX9_VacQ&oe=631DF2E8&_nc_sid=4cb768',
 'non_violating': 1,
 'related_tags': None,
 'subtitle': 'See a few top posts each week',
 'social_context': '',
 'social_context_profile_links': [],
 'social_context_facepile_users': [],
 'follow_button_text': None,
 'show_follow_drop_down': False,
 'formatted_media_count': '11.4K',
 'challenge_id': None,
 'destination_info': None,
 'description': None,
 'debug_info': None,
 'fresh_topic_metadata': None,
 'promo_banner': None,
 'top': {'sec

## 무식한 방법으로 다 찾기

In [100]:
ID_SET = set()
ADDED_SET = set()

In [101]:
keyword = '17년생아들'
url = f'https://www.instagram.com/explore/tags/{keyword}'
browser.get(url)

In [103]:
last_article = browser.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/div/div[1]/div/div/div/div[1]/section/main/article/div/div/div/div[3]/div[3]/a/div[1]/div[2]')

In [104]:
last_article.click()

In [115]:
while True:
    next_button = browser.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div[1]/div/div/div[2]/button')
    next_button.click()
    sleep(5)
    user_id = browser.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[1]/div/header/div[2]/div[1]/div[1]/div/div/div/span/a').text
    ID_SET.add(user_id)
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[1]/div/header/div[2]/div[1]/div[1]/div/div/div/span/a"}
  (Session info: chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x00000001024a9a90 chromedriver + 3889808
1   chromedriver                        0x0000000102438b54 chromedriver + 3427156
2   chromedriver                        0x000000010212a238 chromedriver + 221752
3   chromedriver                        0x000000010215b4c8 chromedriver + 423112
4   chromedriver                        0x0000000102184944 chromedriver + 592196
5   chromedriver                        0x000000010215054c chromedriver + 378188
6   chromedriver                        0x000000010247d2f8 chromedriver + 3707640
7   chromedriver                        0x0000000102480ea8 chromedriver + 3722920
8   chromedri

In [114]:
len(ID_SET)

262